In [ ]:
#| default_exp encoder

In [ ]:
#| hide
#%load_ext autoreload --> Not working TODO:REVISAR
# %autoreload 2 

In [ ]:
#| export
#-- Global
### --- Errors & warnings
import warnings
import traceback
### --- Data & operations
import math
from copy import deepcopy
import pandas as pd
import numpy as np
from fastcore.all import *
from gluonts.dataset.pandas import PandasDataset
import wandb
import time
import einops
## -- plots
import matplotlib.pyplot as plt
## -- Classes & types
from dataclasses import dataclass, field
from typing import List, Optional, Tuple, Callable, Union, Any
#-- Fourier
from tqdm.auto import tqdm
#-- Torch
from torch.nn.modules.loss import _Loss
import torch.profiler as profiler
#-- Dvats
from dvats.memory import *
import dvats.utils as ut
from dvats.config import show_attrdict
from dvats.utils import find_dominant_window_sizes_list
#-- Fastai
from fastai.callback.core import Callback
from fastai.callback.hook import hook_outputs
from fastai.callback.progress import ShowGraphCallback
from fastai.callback.schedule import *
from fastai.callback.tracker import EarlyStoppingCallback, SaveModelCallback
from fastai.callback.wandb import WandbCallback
from fastai.learner import Learner
from fastai.losses import BaseLoss
from fastai.metrics import mae
import fastai.optimizer as fastopt
#-- Tsai
from tsai.data.core import TSDataLoaders
from tsai.data.preparation import SlidingWindow
from tsai.callback.MVP import *
from tsai.imports import *
from tsai.models.InceptionTimePlus import InceptionTimePlus
from tsai.models.explainability import get_acts_and_grads
from tsai.models.layers import *
from tsai.data.validation import combine_split_data, TimeSplitter
from tsai.basics import *
#-- Moirai
import uni2ts
import uni2ts.optim as moirai_opt
import uni2ts.model.moirai.module as moirai
import uni2ts.model.moirai.forecast as moirai_forecast
#| Moment
from momentfm.utils.masking import Masking
from momentfm import MOMENTPipeline
#-- Transformers
from transformers import get_scheduler
#-- Evaluate
import evaluate

/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [ ]:
#| hide
from tsai.all import *
from dvats.imports import beep

# Encoder

> Architectures and functions for creating encoders that create the embeddings

## Encoder class

### Encoder input
Saves the information about the encoder input, including its data, the windows and the meta-data on how to mask it.

In [ ]:
#| export
@dataclass
class EncoderInput:
    # Data
    _data               : Union [ pd.DataFrame, List [ List [ List [ float ]]] ] = None
    _size               : int                               = None
    _shape              : Optional [ Tuple [ int, ... ] ]   = None
    _shapes             : List [ Tuple [ int, ...]]         = None
    stride              : int                               = None
    batch_size          : int                               = None
    _update_size        : bool                              = True
    _update_shape       : bool                              = True
    # Windows                   
    n_windows           : int                               = None
    n_windows_percent   : float                             = None
    validation_percent  : float                             = None
    training_percent    : float                             = None
    window_mask_percent : float                             = None
    # Time                  
    time_flag           : bool                              = None
    
    def __post_init__(self):
        self._update_size       = True
        self._update_shape      = True
        #Todo: check how to validate the input dataset allowing both windowed or not
        # --- Not working
        ###self._data              = ut._check_value(self.data, None, "_data", pd.DataFrame, allow_none = True )
        ###if self._data is None: 
        ###    self._data = ut._validate_nested_list(self._data, None, "_data", [float, int], 3, False, False, False)
        self.stride,_               = ut._check_value(self.stride, 1, "stride", int, positive = True)
        self.batch_size,_           = ut._check_value(self.batch_size, 32, "batch_size", int,  )
        self.validation_percent,_   = ut._check_value(self.validation_percent, 0.2, "validation_percent", percent = True)
        self.training_percent,_     = ut._check_value(self.training_percent, 0.2, "training_percent", percent = True)
        self.window_mask_percent,_  = ut._check_value(self.window_mask_percent, 0.3, "training_percent", percent = True)
        self.time_flag,_            = ut._check_value(self.time_flag, False, "time_flag", bool)

    @property
    def size(self):
        if  ( 
                self._data is not None 
            and ( self._update_size or self._size is None or self._size == 0) 
            and self._data.__class__ != ut.WindowedDataset
        ):
            self._size          = len(self._data)
            self._update_size   = False
            self._size,_ = ut._check_value(self._size, 0, "_size", int)
            self._size = max(self._size, 0)
        elif self._update_size: 
            self._size = 0
            self._update_size = True
        return self._size
    
    @property
    def shape(self) -> Tuple[int, ...]:
        if (
                self._data is not None and 
                ( self._update_shape or self._shape is None or self._shape == 0 ) and
                not self._data.__class__ == ut.WindowedDataset
        ):
            try: 
                self._shape     = self._data.shape
                self._shapes    = [ self._shape ]
            except:
                self._shape  = self._data[0].shape
                self._shapes = [ self._data[i].shape for i in range(len(self._data))]
            self._update_shape = False
        elif self._update_shape: 
            self._shape         = 0
            self._shapes        = []
            self._update_shape  = True
        return self._shape
    @property
    def shapes(self) -> List [ Tuple [ int, ... ]]:
        if (
            self._data is not None and 
            ( self._update_shape or self._shapes is None or self._shapes ==[]) and 
            not self._data.__class__ == ut.WindowedDataset
        ):
            try: 
                self._shape     = self._data.shape
                self._shapes    = [ self._shape ]
            except:
                self._shape     = self._data[0].shape
                self._shapes    = [ self._data[i].shape for i in range(len(self._data))]
            self._update_shape  = False
        elif self._update_shape: 
            self._shape         = 0
            self._shapes        = []
            self._update_shape  = True
        return self._shapes
            
    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        self._data          = value
        self._update_size   = True
        self._update_shape  = True


### LRScheduler
Contains the information needed to use different schedulers types (for MVP, Model and Moirai).

In [ ]:
#| export
@dataclass
class LRScheduler:
    lr              : float = None
    flag            : bool  = None
    name            : str   = None
    num_warmup_steps: int   = None
    scheduler               = None
    monitor         : str   = "valid_loss"
    interval        : str   = "step"

    def __post_init__(self):
        self.lr                 = self._check_lr(self.lr, 1e-5)
        self.flag               = self._check_flag(self.flag, False)
        self.name               = self._check_name(self.name, "OneCycleR")
        self.num_warmup_steps   = self._check_steps(self.num_warmup_steps, 0)

    # Validation methods
    def _check_lr(self, value, default):
        if not isinstance(value, (float, int)) or not math.isfinite(value) or value <= 0:
            warnings.warn(f"Invalid learning rate 'lr' ({value}). Using default: {default}")
            return default
        return float(value)

    def _check_flag(self, value, default):
        if not isinstance(value, bool):
            warnings.warn(f"Invalid type for 'flag' ({type(value)}). Using default: {default}")
            return default
        return value

    def _check_name(self, value, default):
        if not isinstance(value, str):
            warnings.warn(f"Invalid type for 'name' ({type(value)}). Using default: {default}")
            return default
        return value

    def _check_steps(self, value, default):
        if not isinstance(value, int) or value < 0:
            warnings.warn(f"Invalid type or negative value for 'num_warmup_steps' ({value}). Using default: {default}")
            return default
        return value

### EncoderOptimizer
Contains the optimizer and the LRScheduler information.

In [ ]:
#| export
@dataclass
class EncoderOptimizer():
    criterion   : Optional   [ torch.nn.Module ]          = torch.nn.MSELoss
    optimizer   : Optional   [ torch.optim.Optimizer ]    = None
    lr          : Union      [ float, LRScheduler ]       = 1e-5

    def _post__init__(self):
        self.lr,_ = ut._check_value( self.lr, 1e-5, "lr", [ int, float ], False, True, False )

#### Encoder 
Contains the full information about the encoder and its configuration and results

In [ ]:
#| export
@dataclass
class Encoder():
    input               : EncoderInput      = None 
    model               : Union [
                            Learner, 
                            MOMENTPipeline,
                            moirai.MoiraiModule
    ]                                       = None
    mssg                : ut.Mssg           = ut.Mssg()
    cpu                 : bool              = False
    to_numpy            : bool              = False
    num_epochs          : int               = 1
    optim               : EncoderOptimizer  = EncoderOptimizer()
    mask_stateful       : bool              = False
    mask_future         : bool              = False
    mask_sync           : bool              = False
    eval_stats_pre      : AttrDict          = None
    eval_stats_post     : AttrDict          = None
    use_moment_masks    : bool              = False
    model_class         : str               = None
    time_flag           : bool              = False
    use_wandb           : bool              = False
    analysis_mode       : str               = 'online'
    splits              : Tuple             = None
    show_plot           : bool              = False
    norm_by_sample      : bool              = True
    norm_use_single_batch : bool            = True
    #--- Modificar para usar el diccionario
    _metrics             : List [ Union [ Callable, evaluate.EvaluationModule ] ] = None
    _metrics_names       : List [ str ]      = None
    _metrics_args                            = None
    #---
    metrics_dict        : AttrDict          = None
    default_metric                          = MSELossFlat #-- Default for MVP
    criterion                               = None
    scheduler_specific_kwargs : AttrDict    = None # Only for MOIRAI
    #mvp_ws             : Tuple [ int, int ]= 0,0
    errors              : pd.DataFrame      = pd.DataFrame(columns=["window", "error"])
    window_sizes        : List [int]        = None
    best_epoch          : int               = -1
    best_loss           : int               = np.inf
    best_model_state                        = None
    eval_indices_dict   : AttrDict          = None
    ws                  : int               = None #Current window size
    seed                : int               = None
    original_cudnn_benchmark                = None
    original_cudnn_deterministic            = None
    def __post_init__(self):
        self.model          , _ = ut._check_value(self.model, None, "model", [ MOMENTPipeline, Learner, moirai.MoiraiModule ], True, False, False, mssg = self.mssg)
        self.model              = self.set_model_(self.model)
        ## TODO: check how to do this check
        #self.input          , _ = ut._check_value(self.input, EncoderInput(), "input", EncoderInput, True)
        self.mssg           , _ = ut._check_value(self.mssg, ut.Mssg(), "mssg", ut.Mssg, mssg = self.mssg)
        self.cpu            , _ = ut._check_value(self.cpu, False, "cpu", bool, mssg = self.mssg)
        self.to_numpy       , _ = ut._check_value(self.to_numpy, False, "to_numpy", bool,  mssg = self.mssg)
        self.num_epochs     , _ = ut._check_value(self.num_epochs, 1, "num_epochs", int, False, True,  mssg = self.mssg)
        ## TODO: check how to do this check
        #self.optim          , _ = ut._check_value(self.optim, EncoderOptimizer(), "optim", EncoderOptimizer)
        self.mask_stateful  , _ = ut._check_value(self.mask_stateful, False, "mask_statefull", bool,  mssg = self.mssg)
        self.mask_future    , _ = ut._check_value(self.mask_future, False, "mask_future", bool,  mssg = self.mssg)
        self.mask_sync      , _ = ut._check_value(self.mask_sync, False, "mask_sync", bool,  mssg = self.mssg)
        self.eval_stats_pre , _ = ut._check_value(self.eval_stats_pre, None, "eval_stats_pre", AttrDict, True,  mssg = self.mssg)
        self.eval_stats_post, _ = ut._check_value(self.eval_stats_post, None, "eval_stats_post", AttrDict, True,  mssg = self.mssg)
        self.use_moment_masks, _= ut._check_value(self.use_moment_masks, False, "use_moment_masks", bool,  mssg = self.mssg)
        self.model_class        = None # Must be computed through get_model_class to avoid errors
        self.time_flag      , _ = ut._check_value(self.time_flag, False, "time_flag", bool,  mssg = self.mssg)
        self.show_plot      , _ = ut._check_value(self.show_plot, False, "show_plot", bool, mssg = self.mssg)
        self.window_sizes       = [] if self.window_sizes is None else self.window_sizes
        self.seed           , _ = ut._check_value(self.seed, None, "seed", int, True, True, False, self.mssg)
        self.original_cudnn_benchmark       = None
        self.original.cudnn_deterministic   = None
    @property
    def metrics_names(self):
        self._metrics_names = self.metrics_dict.keys() if self.metrics_dict else None
        return self._metrics_names
    
    @property
    def metrics(self):
        self._metrics = None
        if self.metrics_dict:
            self._metrics = [
                self.metrics_dict[n]['metric']
                for n in self.metrics_dict.keys()
            ]
        return self._metrics
    
    @property
    def metrics_args(self):
        self._metric_args = None 
        if self.metrics_dict:
            self._metrics_args = [
                self.metrics_dict[n]['args']
                for n in self.metrics_dict.keys()
            ]
        return self._metrics_args
    
    def print(self, **kwargs):
        self.mssg.print(**kwargs)

    def get_model_class(self, force : bool = False): 
        if force or self.model_class is None:
            self.model_class = str(self.model.__class__)[8:-2]
        return self.model_class
    def set_model_(self, model):
        if model is not None:
            device  = "cpu" if self.cpu else torch.cuda.current_device()
            self.model          = deepcopy(model).to(device)
            self.model_class    = self.get_model_class() 
            try: # Initially it may not be defined and that would result in an execution error
                self.fine_tune_     = self.set_fine_tune_()
            except:
                self.fine_tune_ = None
        return self.model
    def _set_seed_(self, seed: int = 42):
        self._save_cuda_state_()
        self.seed = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic  = True
        #torch.backends.cudnn.benchmark      = False
        #self.mssg.print_error(f"Changing benchmark to False")
        self.mssg.print_error(f"Saving deterministic to True")
    
    def _save_cuda_state_(self):
        self.original_cudnn_benchmark       = torch.backends.cudnn.benchmark
        self.original_cudnn_deterministic   = torch.backends.cudnn.deterministic
        self.mssg.print_error(f"Saving benchmark {self.original_cudnn_benchmark}")
        self.mssg.print_error(f"Saving deterministic {self.original_cudnn_deterministic}")

    def _restore_cuda_(self):
        if self.original_cudnn_benchmark is None or self.original_cudnn_deterministic:
            raise ValueError("Remember to save cuda state using _save_cuda_state_ before trying to restore it.")
        #torch.backends.cudnn.benchmark      = self.original_cudnn_benchmark
        torch.backends.cudnn.deterministic  = self.original_cudnn_deterministic 

    def get_splits_(self, n_sample: int = None):
        self.mssg.initial_(ut.funcname())
        #TODO: add checks for datatype to ensure the dataset is not already windowed
        assert self.analysis_mode in [ 'ofline', 'online'], 'Invalid analysis mode'
        X = self.input.data if n_sample is None else self.input.data[n_sample]
        self.mssg.print(f"len(X)={len(X)}")
        match self.analysis_mode:
            case 'online':
                self.mssg.print("Online analysis", verbose_level = self.mssg.level+1)
                self.splits = TimeSplitter(valid_size = 0.2, show_plot = self.show_plot)(X)
            case 'offline':
                self.mssg.print("Offline analysis", verbose_level = self.mssg.level+1)
                self.splits = get_splits(np.arange(len(X)), valid_size=self.valid_size, show_plot = self.show_plot)
            case _:
                raise NotImplementedError(f"Encoderl{ut.funcname()} | Case {self.analysis_mode} not implemented. Use one of the following options: <online|offline>.")
        self.mssg.print(f"X~{X.shape}")
        self.mssg.print(f"Train: {len(self.splits[0])} | Test { len(self.splits[1])}")
        self.mssg.final()
        return X   
    def setup_scheduler(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def config_optim(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def set_train_and_eval_dataloaders(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def get_acts(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    #--- Moment
    def moment_compute_loss(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def moment_safe_forward_pass(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_eval_preprocess(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_single_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_(self): 
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_train_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_train_loop_step_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_eval_step_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def set_train_size(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def set_train_and_eval_sizes(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def set_train_dataset(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def set_random_dataset(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_mix_windows(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    #--- MVP
    def fine_tune_mvp_single_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_mvp_(self): 
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    #--- Moirai
    def configure_optimizer_moirai(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def get_enc_embs_moirai_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def get_dist_moirai_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moirai_eval_step_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moirai_eval_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moirai_single_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moirai_(self): 
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moirai_train_loop_step_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moirai_train_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    #--- Global
    def fine_tune_single_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def set_fine_tune_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def show_eval_stats(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_eval_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_eval_mix_windows_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")
    def fine_tune_moment_train_mix_windows_(self):
        raise NotImplementedError(f"Encoder.{ut.funcname()} not yet implemented")

In [ ]:
#| export
def set_fine_tune_single_(
    self: Encoder
) -> Callable:
    # Setup mssg
    func = self.mssg.function
    self.mssg.level += 1
    # Go!
    self.mssg.initial_(ut.funcname())
    model_class = self.get_model_class()
    self.mssg.print(f"Model class: {model_class}")
    match model_class:
        case "momentfm.models.moment.MOMENTPipeline":
            self.fine_tune_single_ = self.fine_tune_moment_single_
        case "fastai.learner.Learner":
            self.fine_tune_single_ = self.fine_tune_mvp_single_
        case "uni2ts.model.moirai.module.MoiraiModule":
            self.fine_tune_single_ = self.fine_tune_moirai_single_
        case _:
            self.mssg.print(f"Fine-tune single shot implementation is not yet implemented for {self.model_class}.", verbose_level = self.mssg.level+1)
            raise NotImplementedError(f"fine_tune_single_ | Not yet implemented for {self.model_class}")
    self.mssg.final()
    # Restore mssg
    self.mssg.function = func
    self.mssg.level -= 1
    return(self.fine_tune_single_)
Encoder.set_fine_tune_single_ = set_fine_tune_single_

In [ ]:
#| export
def set_fine_tune_(
    self: Encoder
) -> Callable:
    # Setup mssg
    func = self.mssg.function 
    self.mssg.level += 1
    # Go!
    self.mssg.initial_("set_fine_tune_")
    model_class = self.get_model_class()
    self.mssg.print(f"Model class: {model_class}")
    match model_class:
        case "momentfm.models.moment.MOMENTPipeline":
            self.mssg.print(f"Moment")
            self.fine_tune_ = self.fine_tune_moment_
        case "fastai.learner.Learner":
            self.mssg.print(f"MVP")
            self.fine_tune_ = self.fine_tune_mvp_
        case "uni2ts.model.moirai.module.MoiraiModule":
            raise NotImplementedError(f"fine_tune | Not yet implemented for {self.model_class}")
            self.mssg.print(f"Moirai")
            self.fine_tune_ = self.fine_tune_moirai_
        case _:
            self.mssg.print(f"Fine-tune implementation is not yet implemented for {self.model_class}.", verbose_level = self.mssg.level+1)
            raise NotImplementedError(f"fine_tune | Not yet implemented for {self.model_class}")
    self.mssg.final()
    # Restore mssg
    self.mssg.function = func
    self.mssg.level -= 1
    return(self.fine_tune_)
Encoder.set_fine_tune_ = set_fine_tune_

In [ ]:
#| export
def show_eval_stats(
    self            : Encoder, 
    print_to_path   : bool      = None, 
    print_path      : str       = None, 
    print_mode      : str       = None,
    eval_pre        : bool = False,
    eval_post       : bool = False,
    eval_stats_pre  : AttrDict = None,
    eval_stats_post : AttrDict = None,
    func_name       : str = ""
):
    # No mssg setup/restore as it is a print function
    self.mssg.print(f"Evaluation summary")
    self.eval_stats_pre     = self.eval_stats_pre if eval_stats_pre is None else eval_stats_pre
    self.eval_stats_post    = self.eval_stats_post if eval_stats_post is None else eval_stats_post
    self.mssg.to_path       = self.mssg.to_path if print_to_path is None else print_to_path
    self.mssg.path = self.mssg.path if print_path is None else print_path
    self.mssg.mode = self.mssg.mode if print_mode is None else print_mode        
    if (eval_pre):
        self.mssg.print(f"Eval pre: ")
        show_attrdict(
            self.eval_stats_pre,
            print_to_path   = self.mssg.to_path,
            print_path      = self.mssg.path,
            print_mode      = self.mssg.mode
        )
    if eval_post:
        self.mssg.print(f"Eval post: ")
        show_attrdict(
            self.eval_stats_post,
            print_to_path   = self.mssg.to_path,
            print_path      = self.mssg.path,
            print_mode      = self.mssg.mode 
        )
Encoder.show_eval_stats = show_eval_stats

In [ ]:
#| export
def plot_eval_stats(
    self, 
    figsize     = (10, 6), 
    save_fig    = False, 
    save_path   = "./", 
    fname       = "evaluation_metrics_plot"
):
    """
    Plot evaluation metrics from eval_results_pre and eval_results_post across epochs.
    """
    # Setup mssg
    self.mssg.level += 1
    fname = self.mssg.function
    # Go!
    self.mssg.initial_("plot_eval_stats")
    # Validar que las métricas están presentes
    if not self.eval_stats_pre or not self.eval_stats_post:
        raise ValueError("Evaluation results (eval_stats_pre or eval_stats_post) are missing.")
    
    # Extraer las métricas pre y post
    metrics_pre = self.eval_stats_pre  # Diccionario con las métricas iniciales
    metrics_post = self.eval_stats_post  # Lista de diccionarios con métricas post-época

    # Crear el rango de épocas (incluyendo la inicial)
    epochs = list(range(self.num_epochs-1))
    self.mssg.print(f"epochs~{len(epochs)}: {epochs}")
    # Inicializar el gráfico
    plt.figure(figsize=figsize)
    
    # Iterar sobre las métricas (las claves del diccionario pre/post)
    for metric in metrics_pre.keys():
        # Crear lista de valores para esta métrica
        values = [metrics_pre[metric]] + metrics_post[metric]
        # Graficar la métrica
        plt.plot(epochs, values, label=metric)
    
    # Configurar etiquetas y leyenda
    plt.title("Evaluation Metrics Across Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Metric Value")
    plt.legend()
    plt.grid(True)
    if save_fig:
        # Asegurarse de que el directorio existe
        os.makedirs(save_path, exist_ok=True)
        
        # Ruta completa del archivo
        save_file = os.path.join(save_path, "{fname}.png")
        
        # Guardar el gráfico
        plt.savefig(save_file, dpi=300, bbox_inches="tight")
        self.mssg.print(f"Stats plot saved at: {save_file}")
    plt.show()
    self.mssg.final()
    # Restore mssg
    self.mssg.function = fname
    self.mssg.level -= 1
Encoder.plot_eval_stats = plot_eval_stats

## Architectures

In [ ]:
#|export 
class DCAE_torch(Module):
    def __init__(self, c_in, seq_len, delta, nfs=[64, 32, 12], kss=[10, 5, 5],
                 pool_szs=[2,2,3], output_fsz=10):
        """
        Create a Deep Convolutional Autoencoder for multivariate time series of `d` dimensions,
        sliced with a window size of `w`. The parameter `delta` sets the number of latent features that will be
        contained in the Dense layer of the network. The the number of features
        maps (filters), the filter size and the pool size can also be adjusted."
        """
        assert all_equal([len(x) for x in [nfs, kss, pool_szs]], np.repeat(len(nfs), 3)), \
            'nfs, kss, and pool_szs must have the same length'
        assert np.prod(pool_szs) == nfs[-1], \
            'The number of filters in the last conv layer must be equal to the product of pool sizes'
        assert seq_len % np.prod(pool_szs) == 0, \
            'The product of pool sizes must be a divisor of the window size'
        layers = []
        for i in range_of(kss):
            layers += [Conv1d(ni=nfs[i-1] if i>0 else c_in, nf=nfs[i], ks=kss[i]),
                       nn.MaxPool1d(kernel_size=pool_szs[i])]
        self.downsample = nn.Sequential(*layers)
        self.bottleneck = nn.Sequential(OrderedDict([
            ('flatten', nn.Flatten()),
            ('latent_in', nn.Linear(seq_len, delta)),
            ('latent_out', nn.Linear(delta, seq_len)),
            ('reshape', Reshape(nfs[-1], seq_len // np.prod(pool_szs)))
        ]))
        layers = []
        for i in reversed(range_of(kss)):
            layers += [Conv1d(ni=nfs[i+1] if i != (len(nfs)-1) else nfs[-1],
                              nf=nfs[i], ks=kss[i]),
                       nn.Upsample(scale_factor=pool_szs[i])]
        layers += [Conv1d(ni=nfs[0], nf=c_in, kernel_size=output_fsz)]
        self.upsample = nn.Sequential(*layers)


    def forward(self, x):
        x = self.downsample(x)
        x = self.bottleneck(x)
        x = self.upsample(x)
        return x

In [ ]:
#| hide
foo = torch.rand(3, 1, 48)
m = DCAE_torch(c_in=foo.shape[1], seq_len=foo.shape[2], delta=12)
m(foo).shape

### Dictionary to get the default backbone modules to get the embeddings from

In [ ]:
#| export
ENCODER_EMBS_MODULE_NAME = {
    InceptionTimePlus: 'backbone', # for mvp based models
    DCAE_torch: 'bottleneck.latent_in'#,
    #MoiraiForecast: 'mask_encoding' #TODO: check
    
}

## Callbacks

### Weight & Biases

In [ ]:
#| export
class CustomWandbCallback(WandbCallback):
    """
    Callback that extends WandbCallback to avoid errors when validation 
    is done before training.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def before_fit(self):
        """
        Overrides before_fit to ensure that'epoch' key exists.
        """
        try:
            if wandb.run is not None and wandb.run.step:
                if 'epoch' not in wandb.run.summary._as_dict():
                    wandb.run.summary.update({'epoch': 0})
        except Exception as e:
            print(f"Error while checking epoch in wandb: {e}")
            raise 
        super().before_fit() # Call the original method 

## Get activations

In [ ]:
#| export 
def kwargs_to_gpu_(**kwargs):
    for key in kwargs:
        try: #if not able to be moved, just not move it
            kwargs[key] = kwargs[key].to("cuda")
        except:
            continue
    
def kwargs_to_cpu_(**kwargs):
    for key in kwargs:
        try: #if not able to be moved, just not move it
            kwargs[key] = kwargs[key].cpu()
        except:
            continue
   

In [ ]:
#| export
def get_acts(
    model       : torch.nn.Module, 
    module      : torch.nn.Module, 
    cpu         : bool, 
    verbose     : int               = 0,
    retry       : bool              = False,
    acts_indices: List [ int ]      = None,
    #- Printing options for debugging
    print_to_path   : bool          = False,
    print_path      : str           = "~/data/logs/logs.txt",
    print_mode      : str           = 'a',
    continue_if_fail: bool          = False,
    **model_kwargs #Parameters of the model
):
    if verbose > 0:
        ut.print_flush(f"--> get acts | acts indices: {acts_indices}", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
    if cpu:
        if verbose > 0: ut.print_flush(f"get acts | Moving to cpu", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
        for key in model_kwargs:
            try: #if not able to be moved, just not move it
                model_kwargs[key] = model_kwargs[key].cpu()
            except:
                continue
        model = model.to("cpu")
    else:
        if verbose > 0: ut.print_flush(f"get acts | Moving to gpu", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
        for key in model_kwargs:
            try: #if not able to be moved, just not move it
                model_kwargs[key] = model_kwargs[key].to("cuda")
            except:
                continue
        model = model.to("cuda")
    if verbose > 0: ut.print_flush(f"get acts | Add hooks", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
    h_act = hook_outputs([module], detach = True, cpu = cpu, grad = False)
    with torch.no_grad():
        if verbose > 0: ut.print_flush(f"get acts | --> Run forward", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
        if retry:
            if verbose > 0: ut.print_flush(f"get acts | Retry", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
            try: 
                preds = model.eval()(**model_kwargs)
            except Exception as e:
                ut.print_flush(f"get acts | Retry | Error: {e}", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                ut.print_flush(f"get acts | Retry | Kwargs: {model_kwargs}", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                if not cpu:
                    ut.print_flush(f"get acts | Retry | Moving to cpu", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                    for key in model_kwargs:
                        try: #if not able to be moved, just not move it
                            model_kwargs[key] = model_kwargs[key].cpu()
                        except:
                            continue
                    model.to("cpu")
                    if verbose > 0: ut.print_flush(f"get acts | Retry | cpu", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                    ut.print_flush(f"get acts | Retry | Get acts", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                    preds = model.eval()(**model_kwargs)
        else:
            if verbose > 2: ut.print_flush(f"get acts | No Retry | Get acts | model kwargs: {model_kwargs}", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
            preds = model.eval()(**model_kwargs)
    if acts_indices is None:
        res = [o.stored for o in h_act]
    else: 
        stored = [o.stored for o in h_act]
        res = [stored[i] for i in acts_indices]
        if len(acts_indices) == 1:
            res = res[0]
        del stored
    if verbose > 0: ut.print_flush(f"get acts | Run forward -->", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
    if verbose > 0:ut.print_flush(f"get acts -->", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
    return res

In [ ]:
#| export
def encoder_get_acts(
    self        : Encoder,
    module      : torch.nn.Module, 
    retry       : bool              = False,
    acts_indices: List [ int ]      = None,
    continue_if_fail: bool          = False,
    **model_kwargs #Parameters of the model
):
    # Setup mssg 
    self.mssg.level += 1
    func = self.mssg.function 
    self.mssg.initial_(ut.funcname())
    # Go!
    self.print(f"acts indices: {acts_indices}")
    cpu = self.cpu == "cpu"
    if cpu:
        self.mssg.print(f"Moving to cpu")
        for key in model_kwargs:
            try: #if not able to be moved, just not move it
                model_kwargs[key] = model_kwargs[key].cpu()
            except:
                continue
        model = model.to("cpu")
    else:
        self.mssg.print(f"Moving to gpu")
        for key in model_kwargs:
            try: #if not able to be moved, just not move it
                model_kwargs[key] = model_kwargs[key].to("cuda")
            except:
                continue
        model = model.to("cuda")
    self.mssg.print(f"Add hooks")
    h_act = hook_outputs([module], detach = True, cpu = cpu, grad = False)
    with torch.no_grad():
        self.mssg.print(f"--> Run forward")
        if retry:
            self.mssg.print(f"Retry")
            try: 
                preds = model.eval()(**model_kwargs)
            except Exception as e:
                self.mssg.print(f"Retry | Error: {e}")
                self.mssg.print(f"Retry | Kwargs: {model_kwargs}")
                if not cpu:
                    self.mssg.print(f"Retry | Moving to cpu")
                    for key in model_kwargs:
                        try: #if not able to be moved, just not move it
                            model_kwargs[key] = model_kwargs[key].cpu()
                        except:
                            continue
                    model.to("cpu")
                    self.mssg.print(f"Retry | cpu")
                    self.mssg.print(f"Retry | Get acts")
                    preds = model.eval()(**model_kwargs)
        else:
            self.mssg.print(f"No Retry | Get acts | model kwargs: {model_kwargs}")
            preds = model.eval()(**model_kwargs)
    if acts_indices is None:
        res = [o.stored for o in h_act]
    else: 
        stored = [o.stored for o in h_act]
        res = [stored[i] for i in acts_indices]
        if len(acts_indices) == 1:
            res = res[0]
        del stored
    self.mssg.print(f"get acts | Run forward -->")
    self.mssg.final()
    # Restore mssg 
    self.mssg.function = func
    self.mssg.level -= 1
    return res

Encoder.get_acts = encoder_get_acts

In [ ]:
#| export
# TODO: Simplificar & meter en class Encoder
def get_acts_moment(
    enc_learn, 
    cpu             : bool          = False, 
    verbose         : int           = 0, 
    y               : List [ float ]= [], 
    mask                            = None, 
    padd_step       : int           = 100, 
    # Parameters for avoiding errors
    retry           : bool          = False, 
    max_trials      : int           = 5,
    # Activation selector (various vectors in the acts)
    acts_indices    : List [ int ]  = [0],
    #- Printing options for debugging
    print_to_path   : bool          = False,
    print_path      : str           = "~/data/logs/logs.txt",
    print_mode      : str           = 'a',
    continue_if_fail: bool          = False
):
    success = False 
    trial = 0
    embs = None
    while not success and trial < max_trials:
        trial += 1
        try:
            if verbose > 0: ut.print_flush(f"get_acts_moment | Trial {trial} | x_enc ~ {y.shape}", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
            embs = get_acts(
                model = enc_learn,
                #module = enc_learn.encoder.dropout,
                module = enc_learn.head.dropout,
                cpu = cpu,
                verbose = 0,
                x_enc = y,
                retry = retry,
                acts_indices = acts_indices,
                mask = mask,
                print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, print_time = print_to_path
            )
            success = True
            if verbose > 0 and acts_indices == [0] : ut.print_flush(f"get_acts_moment | Trial {trial} | embs ~ {embs.shape}", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
        except Exception as e:
            if trial == max_trials - 1 : raise
            if verbose > 0:
                ut.print_flush(f"get_acts_moment | Trial {trial} | About to pad X (encoder input) | exception {e} | padd step: {padd_step}", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                ut.print_flush(f"get_acts_moment | Trial {trial} | y ~ {y.shape}", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
            if "tensor a" in str(e) and "tensor b" in str(e):
                match = re.search(r'tensor a \((\d+)\) must match the size of tensor b \((\d+)\)', str(e))
                tensor_a_size = int(match.group(1))
                tensor_b_size = int(match.group(2))
                padd = True
                if trial > 1: 
                    if verbose > 0: ut.print_flush(f"------------------- Trial {trial}  -----------------", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                    if tensor_a_size > tensor_a_size_old:
                        if verbose > 0:  ut.print_flush(f"------------------- Trial {trial} | a > a_old -----------------", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                        padd = False
                        y = y [ ..., : tensor_a_size - tensor_b_size]
                        if verbose > 0: ut.print_flush(f"------------------- Trial {trial} |a > a_old | Reduced |  y ~ {y.shape} -----------------", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                if padd:
                    if verbose > 0: ut.print_flush(f"------------------- Trial {trial} | Padd -----------------", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                    if tensor_a_size > tensor_b_size: 
                        if verbose > 0: ut.print_flush(f"------------------- Trial {trial} | Padd | a > b -----------------", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                        padd_step = tensor_a_size - tensor_b_size
                    y = torch.nn.functional.pad(y,(0,padd_step))
                tensor_a_size_old = tensor_a_size
            else:
                if verbose > 0: ut.print_flush("Not the usual error. No padding, just fail", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, verbose = verbose, print_time = print_to_path)
                raise
    return embs

In [ ]:
#| export
def moment_safe_forward_pass(
    self           : Encoder,
    batch, 
    input_mask              = None, 
    mask                    = None, 
    register_errors : bool  = True,
    continue_if_fail: bool  = True
): 
    # mssg configuration
    func    = self.mssg.function 
    self.mssg.level += 1
    self.mssg.initial_(ut.funcname())
    # Move all tensors to the device
    device  = "cpu" if self.cpu else torch.cuda.current_device()
    self.mssg.print(f"cpu | {self.cpu} | device | {device}")

    if input_mask is not None: input_mask = input_mask.to(device)
    if mask is not None: mask = mask.to(device)
    batch       = batch.to(device)
    self.model  = self.model.to(device)
    # Checking values depending on the verbosity level
    self.mssg.print(f"Device {device} | input_mask~{input_mask.shape} device: {input_mask.device if input_mask is not None else 'None'}")
    self.mssg.print(f"Device {device} | mask device~{mask.shape}: {mask.device if mask is not None else 'None'}")
    self.mssg.print(f"Device {device} | batch~{batch.shape} device: {batch.device}")
    # Compute the output and raise or not errors depending on the configuration for the training loop
    success = False
    try:
        #--- Añadidos comentarios para revisar el uso de CUDA (quitar el _error y aumentar verbose cuando se termine la depuración)
        self.mssg.print(f"Using device: {torch.cuda.current_device()}")
        self.mssg.print(f"CUDA is available: {torch.cuda.is_available()}")
        self.mssg.print(f"Model device: {next(self.model.parameters()).device}")
        self.mssg.print(f"Devices: batch - {batch.device} | input_mask - {batch.device} | mask - {mask.device}.")
        #---
        batch       = batch.float()
        input_mask  = input_mask.float()
        mask        = mask.float()
        output  = self.model(
            x_enc       = batch, 
            input_mask  = input_mask, 
            mask        = mask
        )
        success = True
    except Exception as e:        
        traceback.print_exc()
        self.mssg.print("Error computing Moment forward pass.")
        success = False
        if not continue_if_fail: raise ValueError("Error computing Moment forward pass.")
    # Print error message if failed and continue_if_fail = True
    if not success:
        mssg = "Error computing safe moment forward pass."
        self.mssg.print_error(mssg)
        self.mssg.print_error(f"Device {device} | input_mask~{input_mask.shape} device: {input_mask.device if input_mask is not None else 'None'}")
        self.mssg.print_error(f"Device {device} | mask device~{mask.shape}: {mask.device if mask is not None else 'None'}")
        self.mssg.print_error(f"Device {device} | batch~{batch.shape} device: {batch.device}")
        traceback.print_exc()
        if not continue_if_fail: raise ValueError(mssg)#No debería llegar aquí
    else: 
       self.mssg.print(f"Computation done | output is None? {output is None}")
    # --- Restore mssg
    self.mssg.final()
    self.mssg.level     -= 1
    self.mssg.function  = func
    return output, success

Encoder.moment_safe_forward_pass = moment_safe_forward_pass

## Getting the embeddings (activations) from the encoder
TODO: Añadir a la clase Encoder un get_enc_embs y los setters asociados similares a los de fine_tune 

In [ ]:
#| export
def get_enc_embs_ensure_batch_size_(
    dls        : TSDataLoaders,
    batch_size : int = None,
    verbose    : int = 0
) -> None:
    if batch_size is None:
        if verbose > 1: 
            ut.print_flush(f"[ Get Encoder Embeddings Ensure Batch Size ] No batch size proposed", verbose = verbose)
        if dls.bs == 0: 
            if verbose > 1: 
                ut.print_flush(f"[ Get Encoder Embeddings Ensure Batch Size ] Using value 64 as 0 is not a valid value.", verbose = verbose)
            enc_learn.dls.bs = 64
        elif verbose > 1: 
            ut.print_flush(f"[ Get Encoder Embeddings Ensure Batch Size ] Using the original value: {dls.bs}", verbose = verbose)
    else:
        dls.bs = batch_size
        if verbose > 1: 
            ut.print_flush(f"[ Get Encoder Embeddings Ensure Batch Size ] Batch size proposed. Using {dls.bs}", verbose = verbose)

In [ ]:
#| export
def get_enc_embs_MVP(
    X               : List [ List [ List [ float ] ] ], 
    enc_learn       : Learner, 
    module          : str  = None, 
    cpu             : bool = False, 
    average_seq_dim : bool = True, 
    to_numpy        : bool = True,
    batch_size      : int  = None,
    verbose         : int  = 0
):
    """
        Get the embeddings of X from an encoder, passed in `enc_learn as a fastai
        learner. By default, the embeddings are obtained from the last layer
        before the model head, although any layer can be passed to `model`.
        Input
        - `cpu`: Whether to do the model inference in cpu of gpu (GPU recommended)
        - `average_seq_dim`: Whether to aggregate the embeddings in the sequence dimensions
        - `to_numpy`: Whether to return the result as a numpy array (if false returns a tensor)
        - `batch_size`: force data loader to use the input batch size
        - `verbose`: print flag. More big, more information.
    """
    
    if cpu:
        if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] CPU")
        enc_learn.dls.cpu()
        enc_learn.cpu()
    else:
        if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] --> GPU")
        if verbose > 1: ut.print_flush("[ Get Encoder Embeddings ] GPU | Ensure empty cache")
        torch.cuda.empty_cache()
        if verbose > 1: ut.print_flush("[ Get Encoder Embeddings ] GPU | Move & exec into CUDA")
        enc_learn.dls.cuda()
        enc_learn.cuda()
        if torch.cuda.is_available():
            if verbose > 1: 
                ut.print_flush("[ Get Encoder Embeddings ] GPU | CUDA is available")
                ut.print_flush(f"[ Get Encoder Embeddings ] GPU | CUDA is available | current device id {torch.cuda.current_device()}")
                ut.print_flush(f"[ Get Encoder Embeddings ] GPU | CUDA is available | current device name {torch.cuda.get_device_name(torch.cuda.current_device())}")            
        else:
            if verbose > 1: ut.print_flush("[ Get Encoder Embeddings ] GPU | CUDA is not available")
        if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] GPU -->")

    #if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] Ensure the correct batch size")
    #get_enc_embs_ensure_batch_size_(enc_learn.dls, batch_size, verbose)
    
    if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] Set dataloader from X (enc_learn does not contain dls)")
    aux_dl = enc_learn.dls.valid.new_dl(X=X)
    get_enc_embs_ensure_batch_size_(aux_dl, batch_size, verbose)
    if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] Get module")
    module = nested_attr(enc_learn.model,ENCODER_EMBS_MODULE_NAME[type(enc_learn.model)]) if module is None else module
    
    if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] get_acts_and_grads ")
    if verbose > 1: ut.print_flush(f"[ Get Encoder Embeddings ] get_acts_and_grads bs = {aux_dl.bs}")
    
    embs = [
        get_acts_and_grads(
            model   = enc_learn.model,
            modules = module,
            x       = xb[0], 
            cpu     = cpu
        )[0] 
        for xb in aux_dl
    ]
    if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | --> Concat")
    if not cpu:
        if verbose > 1: ut.print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | Check neccesary & free memory")
        total_emb_size = sum([emb.element_size() * emb.nelement() for emb in embs])
        free_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()
        if (total_emb_size < free_memory):
            if verbose > 1: ut.print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | Check neccesary & free memory | Fits in GPU -> Computing in GPU")
            embs=[emb.cuda() for emb in embs]
        else:
            if verbose > 1: ut.print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | Check neccesary & free memory | Does not fit in GPU -> Computing in CPU")
            embs=[emb.cpu() for emb in embs]
    if verbose > 1: ut.print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | to_concat")
    embs = to_concat(embs)
    if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat -->")
    
    if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] Reduce to 2 dimensions.")
    if embs.ndim == 3 and average_seq_dim: embs = embs.mean(axis=2)
    if verbose > 0: ut.print_flush("[ Get Encoder Embeddings ] Ensure CPU saving & numpy format")
    if to_numpy: embs = embs.numpy() if cpu else embs.cpu().numpy()
    return embs

In [ ]:
#| export

def get_enc_embs_MVP_set_stride_set_batch_size(
    X                  : List [ List [ List [ float ] ] ], 
    enc_learn          : Learner, 
    stride             : int, 
    batch_size         : int, 
    module             : str  = None, 
    cpu                : bool = False, 
    average_seq_dim    : bool = True, 
    to_numpy           : bool = True, 
    verbose            : int  = 0, 
    time_flag          : bool = False, 
    chunk_size         : int  = 0, 
    check_memory_usage : bool = False
):
    """
        Get the embeddings of X from an encoder, passed in `enc_learn as a fastai
        learner. By default, the embeddings are obtained from the last layer
        before the model head, although any layer can be passed to `model`.
        Input
        - `X`: encoder input
        - `enc_learn`: trained encoder
        - `stride`: stride used for the training. Neccesary for adjusting the encoder input
        - `batch_size`: value to force the dataloader to use.
        - `module`: for geting the embeddings of an specific layer.
        - `cpu`: Whether to do the model inference in cpu of gpu (GPU recommended)
        - `average_seq_dim`: Whether to aggregate the embeddings in the sequence dimensions
        - `to_numpy`: Whether to return the result as a numpy array (if false returns a tensor)
        - `verbose`: For printing messages. More big, more messages.
        - `time_flag`: To take note of the execution time required by this function
        - `chunk_size`: For spliting the embedings reading in batches of `chunk_size` size.
        - `check_memory_usage`: For showing messages of the current state of the memory.
    """
    if time_flag:
        t_start = time.time()
    if verbose > 0:
        ut.print_flush("--> get_enc_embs_MVP_set_stride_set_batch_size", verbose = verbose)
    if check_memory_usage: gpu_memory_status()
    X = X[::stride]
    enc_learn.dls.bs = batch_size 

    get_enc_embs_ensure_batch_size_(enc_learn.dls, batch_size, verbose)
    
    if verbose > 0: ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Check CUDA | X ~ {X.shape[0]}", verbose = verbose)
    if cpu:
        if verbose > 0: ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size | Get enc embs CPU")
        enc_learn.dls.cpu()
        enc_learn.cpu()
    else:
        if torch.cuda.is_available():
            if verbose > 0: 
                ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | CUDA device id: {torch.cuda.current_device()}", verbose = verbose)
                ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | CUDA device name: {torch.cuda.get_device_name(torch.cuda.current_device())}", verbose = verbose)
                ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Ensure empty cache & move 2 GPU", verbose = verbose)
            torch.cuda.empty_cache()
            enc_learn.dls.cuda()
            enc_learn.cuda()
        else:
            if verbose > 0: ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size | No cuda available. Set CPU = true")
            cpu = True
            
    get_enc_embs_ensure_batch_size_(enc_learn.dls, batch_size, verbose)

    if verbose > 0: ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size | Set dataset from X (enc_learn does not contain dls)", verbose = verbose)
    aux_dl = enc_learn.dls.valid.new_dl(X=X)
    aux_dl.bs = enc_learn.dls.bs if enc_learn.dls.bs>0 else 64
    if verbose > 0: ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size | Get module", verbose = verbose)
    module = nested_attr(enc_learn.model,ENCODER_EMBS_MODULE_NAME[type(enc_learn.model)]) if module is None else module
    
    if verbose > 0: 
        #ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | module ", module)
        ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | aux_dl len {len(aux_dl)}", verbose = verbose)
        ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | aux_dl.batch_len {len(next(iter(aux_dl)))}", verbose = verbose)
        ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | aux_dl.bs {aux_dl.bs}", verbose = verbose)
        if (not cpu):
            total = torch.cuda.get_device_properties(device).total_memory
            used = torch.cuda.memory_allocated(torch.cuda.current_device())
            reserved = torch.cuda.memory_reserved(torch.cuda.current_device())
            ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | total_mem {total}", verbose = verbose)
            ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | used_mem {used}", verbose = verbose)
            ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | reserved_mem {reserved}" ,verbose = verbose)
            ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | available_mem {total-reserved}", verbose = verbose)
            sys.stdout.flush()
                                              
    if (cpu or ( chunk_size == 0 )):
        embs = [
            get_acts_and_grads(
                model=enc_learn.model,
                modules=module, 
                x=xb[0], 
                cpu=cpu
            )[0] 
            for xb in aux_dl
        ]
        if not cpu: embs=[emb.cpu() for emb in embs]
    else:
        embs = []
        total_chunks=max(1,round(len(X)/chunk_size))
        if verbose > 0: ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | aux_dl len | {str(len(X))}  chunk size: {str(chunk_size) } => { str(total_chunks) }  chunks", verbose = verbose)
        for i in range(0, total_chunks):
            if verbose > 0: 
                ut.print_flush(f"get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | Chunk [ {str(i)}/{str(total_chunks)}] => {str(round(i*100/total_chunks))}%", verbose = verbose)
                sys.stdout.flush()
            chunk = [batch for (n, batch) in enumerate(aux_dl) if (chunk_size*i <= n  and chunk_size*(i+1) > n) ]
            chunk_embs = [
                get_acts_and_grads(
                    model=enc_learn.model,
                    modules=module,
                    x=xb[0], 
                    cpu=cpu
                )[0]
                for xb in chunk
            ]
            # Mueve los embeddings del bloque a la CPU
            chunk_embs = [emb.cpu() for emb in chunk_embs]
            embs.extend(chunk_embs)
            torch.cuda.empty_cache()
        if verbose > 0: 
            ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size | Get acts and grads | 100%", verbose = verbose)
            sys.stdout.flush()
    
    if verbose > 0: ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size | concat embeddings", verbose = verbose)
    
    embs = to_concat(embs)
    
    if verbose > 0: ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size | Reduce", verbose = verbose)
    
    if embs.ndim == 3 and average_seq_dim: embs = embs.mean(axis=2)
    
    if verbose > 0: ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size | Convert to numpy", verbose = verbose)
    
    if to_numpy: 
        if cpu or chunk_size > 0:
            embs = embs.numpy() 
        else: 
            embs = embs.cpu().numpy()
            torch.cuda.empty_cache()
    if time_flag:
        t = time.time()-t_start
        if verbose > 0:
            ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size " + str(t) + " seconds -->", verbose = verbose)
        else:
            ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size " + str(t) + " seconds", verbose = verbose)
    if check_memory_usage: gpu_memory_status()
    if verbose > 0: 
        ut.print_flush("get_enc_embs_MVP_set_stride_set_batch_size -->", verbose = verbose)
    return embs

In [ ]:
#| export
def get_enc_embs_moment(
    X               : List [ List [ List [ float ] ] ], 
    enc_learn       : Learner, 
    cpu             : bool = False, 
    to_numpy        : bool = True,
    verbose         : int  = 0,
    average_seq_dim : bool = True
):
    if verbose > 0: 
        ut.print_flush("--> get_enc_embs_moment", verbose = verbose)
    # Move tensor and model to GPU
    if cpu or not torch.cuda.is_available():
        if verbose > 0: 
            ut.print_flush("get_enc_embs_moment | Using CPU (maybe no cuda available)", verbose = verbose)
        cpu = True
        enc_learn.cpu()
    else:
        if verbose > 0: 
            ut.print_flush("get_enc_embs_moment | Using CUDA", verbose = verbose)
        enc_learn = enc_learn.to("cuda")
    if verbose > 0: ut.print_flush("get_enc_embs_moment | Convert y", verbose = verbose)
    enc_learn.eval()
    if cpu:
        y = torch.from_numpy(X).cpu().float()
    else:
        y = torch.from_numpy(X).to("cuda").float()
    # Get output
    with torch.no_grad():
        if verbose > 0: 
            ut.print_flush("get_enc_embs_moment | Get outputs", verbose = verbose)
        outputs = enc_learn(y)
        if verbose > 0:
            ut.print_flush(f"get_enc_embs_moment | Final shape: X ~ {y.shape}", verbose = verbose)
                
    #| move tensors and models back to CPU
    if not cpu:
        y = y.detach().cpu().numpy()
    if verbose > 0: 
        ut.print_flush("get_enc_embs_moment | Get Embeddings", verbose = verbose)
    embeddings = outputs.embeddings.detach().cpu()
    if average_seq_dim: 
        embeddings = embeddings.mean(dim = 1)
    if to_numpy:
        embeddings = embeddings.cpu().numpy()
    if verbose > 0: 
        ut.print_flush("get_enc_embs_moment -->", verbose = verbose)
    return embeddings

In [ ]:
#| export
def get_enc_embs_moment_reconstruction(
    X               : List [ List [ List [ float ] ] ], 
    enc_learn       : Learner, 
    cpu             : bool          = False, 
    to_numpy        : bool          = True,
    verbose         : int           = 0,
    average_seq_dim : bool          = True,
    padd_step       : int           = 2,
    #- Printing options for debugging
    print_to_path   : bool          = False,
    print_path      : str           = "~/data/logs/logs.txt",
    print_mode      : str           = 'a',
    continue_if_fail: bool          = False
):
    """
    For reconstruction sometimes mask get invalid values
    To avoid them, the last dimension (sequence length) is padded with 0's until the error is skippedd
    It should only get one iteration as it seems to be some MOMENT internal configuration for patches.
    """
    if cpu:
        enc_learn = enc_learn.cpu()
        y = torch.from_numpy(X).cpu().float()
    else:
        enc_learn = enc_learn.to("cuda")
        y = torch.from_numpy(X).to("cuda").float()
    embs = get_acts_moment(
        enc_learn       = enc_learn, 
        cpu             = cpu, 
        verbose         = verbose, 
        y               = y, 
        mask            = None,
        padd_step       = padd_step,
        retry           = False ,
        max_trials      = 5,
        print_to_path   = print_to_path, print_path = print_path, print_mode = print_mode
    )
    if average_seq_dim: 
        embs = embs.mean(dim = 1).mean(dim = 1)
    if to_numpy:
        embs = embs.cpu().numpy()
    return embs

---> TODO: averiguar de qué module salen realmente los embeddings y usar el get_acts_and_grads como en MVP <---

In [ ]:
#| export
def get_past_target_moirai(
    enc_input   : List [ List [ List [ Float ] ] ]
):
    return einops.rearrange(
        torch.as_tensor(enc_input, dtype = torch.float32),
        "n_windows n_vars window_size -> n_windows window_size n_vars"
    )

def get_forecast_model_moirai(
    module,
    past_target,
    patch_size,
):
    return moirai_forecast.MoiraiForecast(
        module                      = module,
        prediction_length           = past_target.shape[2],
        context_length              = past_target.shape[1],
        patch_size                  = patch_size,
        num_samples                 = 100,
        target_dim                  = past_target.shape[2],
        feat_dynamic_real_dim       = 0,
        past_feat_dynamic_real_dim  = 0
    )

In [ ]:
#| export
def get_enc_embs_moirai(
    enc_input       : List [ List [ List [ Float ] ] ], 
    enc_model       : moirai.MoiraiModule, 
    cpu             : False,
    average_seq_dim : bool = True, 
    verbose         : int  = 0,
    to_numpy        : bool = True,
    patch_size      : int  = 8,
    time            : bool = False,
    return_kwargs   : bool = False
):
    mssg = ut.Mssg()
    if time: 
        timer = ut.Time(mssg = mssg)
        timer.start()
    if verbose > 0: 
        ut.print_flush("--> get_enc_embs_moirai", verbose = verbose)
    # Move tensor and model to GPU
    past_target = get_past_target_moirai(enc_input)
    if cpu or not torch.cuda.is_available():
        if verbose > 0: ut.print_flush("get_enc_embs_moirai | Using CPU (maybe no cuda available)", verbose = verbose)
        cpu = True
        enc_model.cpu()
        past_target.cpu()
    else:
        if verbose > 0: ut.print_flush("get_enc_embs_moirai | Using CUDA", verbose = verbose)
        enc_model   = enc_model.to("cuda")
        past_target = past_target.to("cuda")
        
    if verbose > 0: ut.print_flush("get_enc_embs_moirai | Get Outputs", verbose = verbose)

    
    past_observed_target = torch.ones_like(past_target, dtype=torch.bool)
    past_is_pad = torch.zeros_like(past_target, dtype=torch.bool)[...,:,-1] # Kill last dimension

    if (verbose > 1):
        ut.print_flush(f"--> get_enc_embs_moirai | past_target ~ {past_target.shape}")
        ut.print_flush(f"--> get_enc_embs_moirai | past_observed_target ~ {past_observed_target.shape}")
        ut.print_flush(f"--> get_enc_embs_moirai | past_is_pad ~ {past_is_pad.shape}")
        ut.print_flush(f"--> get_enc_embs_moirai | Auxiliar model")
        ut.print_flush(f"--> get_enc_embs_moirai | Auxiliar model | Before Memory:")
        gpu_memory_status()
    
    # Auxiliar model for conversions just to ensure correct sizes
    #not neccesary, is the same module initially downloaded...
    #module = moirai.MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-small")
    
    forecast_model =  moirai_forecast.MoiraiForecast(
        module=enc_model,
        prediction_length=past_target.shape[2], #random, just for getting the model
        context_length=past_target.shape[1],
        patch_size=patch_size,
        num_samples=100, #Random, is the number of forecasting, not interesting for us
        target_dim=past_target.shape[2],
        feat_dynamic_real_dim=0,
        past_feat_dynamic_real_dim=0,
    )
    
    if verbose > 0:
        ut.print_flush(f"--> get_enc_embs_moirai | Auxiliar model | After Memory:")
        gpu_memory_status()
        ut.print_flush(f"--> get_enc_embs_moirai | Convert sizes")
    (
    target,
    observed_mask,
    sample_id,
    time_id,
    variate_id,
    prediction_mask,
    ) = forecast_model._convert(
        patch_size,
        past_target,
        past_observed_target,
        past_is_pad
    )
    if verbose > 1:
        ut.print_flush(f"get_enc_embs_moirai | target ~ {target.shape}")
        ut.print_flush(f"get_enc_embs_moirai | observed_mask ~ {observed_mask.shape}")
        ut.print_flush(f"get_enc_embs_moirai | sample_id ~ {sample_id.shape}")
        ut.print_flush(f"get_enc_embs_moirai | time_id ~ {time_id.shape}")
        ut.print_flush(f"get_enc_embs_moirai | variate_id ~ {variate_id.shape}")
        ut.print_flush(f"get_enc_embs_moirai | prediction_mask ~ {prediction_mask.shape}")
        gpu_memory_status()
    forecast_model = None
    if not cpu: torch.cuda.empty_cache()
    if verbose > 0:
        ut.print_flush(f"--> get_enc_embs_moirai | Delete Auxiliar model | After Memory:")
        gpu_memory_status()
    
    model_kwargs={
        'target': target, 
        'observed_mask': observed_mask,
        'sample_id': sample_id,
        'time_id': time_id,
        'variate_id': variate_id,
        'prediction_mask': prediction_mask,
        'patch_size': torch.ones_like(sample_id, dtype = torch.float32)*patch_size
    } 
    if verbose > 0: 
        ut.print_flush(f"get_enc_embs_moirai | About to get activations")
    acts = get_acts(
        model  = enc_model, 
        module = enc_model.encoder.norm, 
        cpu    = cpu,
        verbose = verbose,
        retry = True,
        acts_indices = [0],
        **model_kwargs #Parameters of the model
    )
    
    embs = acts
    acts = None
    if average_seq_dim :
        if verbose > 0: 
            ut.print_flush(f"get_enc_embs_moirai | About to reduce activations", verbose = verbose)
        embs = embs.mean(dim = 1)
    
    if not cpu:
        #ut.print_flush(f"get_enc_embs_moirai | enc_input to cpu")
        #enc_input.cpu()
        ut.print_flush(f"get_enc_embs_moirai | enc_model to cpu", verbose = verbose)
        enc_model.cpu()
        ut.print_flush(f"get_enc_embs_moirai | torch cuda empty cache", verbose = verbose)
        torch.cuda.empty_cache()
    if to_numpy: 
        if cpu > 0:
            embs = embs.numpy() 
        else: 
            embs = embs.cpu().numpy()
            torch.cuda.empty_cache()
    if verbose > 0: 
        ut.print_flush(f"get_enc_embs_moirai | embs ~ {embs.shape}", verbose = verbose)
        ut.print_flush("get_enc_embs_moirai -->", verbose = verbose)
    if return_kwargs:
        return embs, model_kwargs
    return embs

In [ ]:
#| export
def get_dist_moirai(
    enc_input       : List [ List [ List [ Float ] ] ], 
    enc_model       : moirai.MoiraiModule, 
    cpu             : False,
    verbose         : int  = 0,
    patch_size      : int  = 8,
    time            : bool = False,
    return_kwargs   : bool = False
):
    mssg = ut.Mssg()
    if time: 
        timer = ut.Time(mssg = mssg)
        timer.start()
    if verbose > 0: 
        ut.print_flush("--> get_dist_moirai", verbose = verbose)
    # Move tensor and model to GPU
    past_target = get_past_target_moirai(enc_input)
    if cpu or not torch.cuda.is_available():
        if verbose > 0: ut.print_flush("get_dist_moirai | Using CPU (maybe no cuda available)", verbose = verbose)
        cpu = True
        enc_model.cpu()
        past_target.cpu()
    else:
        if verbose > 0: ut.print_flush("get_dist_moirai | Using CUDA", verbose = verbose)
        enc_model.to("cuda")
        past_target.to("cuda")
        
    if verbose > 0: ut.print_flush("get_dist_moirai | Get Outputs", verbose = verbose)

    
    past_observed_target = torch.ones_like(past_target, dtype=torch.bool)
    past_is_pad = torch.zeros_like(past_target, dtype=torch.bool)[...,:,-1] # Kill last dimension

    if (verbose > 1):
        ut.print_flush(f"--> get_dist_moirai | past_target ~ {past_target.shape}")
        ut.print_flush(f"--> get_dist_moirai | past_observed_target ~ {past_observed_target.shape}")
        ut.print_flush(f"--> get_dist_moirai | past_is_pad ~ {past_is_pad.shape}")
        ut.print_flush(f"--> get_dist_moirai | Auxiliar model")
        ut.print_flush(f"--> get_dist_moirai | Auxiliar model | Before Memory:")
        gpu_memory_status()
    
    # Auxiliar model for conversions just to ensure correct sizes
    #not neccesary, is the same module initially downloaded...
    #module = moirai.MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-small")
    
    forecast_model =  moirai_forecast.MoiraiForecast(
        module=enc_model,
        prediction_length=past_target.shape[2], #random, just for getting the model
        context_length=past_target.shape[1],
        patch_size=patch_size,
        num_samples=100, #Random, is the number of forecasting, not interesting for us
        target_dim=past_target.shape[2],
        feat_dynamic_real_dim=0,
        past_feat_dynamic_real_dim=0,
    )
    
    if verbose > 0:
        ut.print_flush(f"--> get_dist_moirai | Auxiliar model | After Memory:")
        gpu_memory_status()
        ut.print_flush(f"--> get_dist_moirai | Convert sizes")
    (
    target,
    observed_mask,
    sample_id,
    time_id,
    variate_id,
    prediction_mask,
    ) = forecast_model._convert(
        patch_size,
        past_target,
        past_observed_target,
        past_is_pad
    )
    if verbose > 1:
        ut.print_flush(f"get_dist_moirai | target ~ {target.shape}")
        ut.print_flush(f"get_dist_moirai | observed_mask ~ {observed_mask.shape}")
        ut.print_flush(f"get_dist_moirai | sample_id ~ {sample_id.shape}")
        ut.print_flush(f"get_dist_moirai | time_id ~ {time_id.shape}")
        ut.print_flush(f"get_dist_moirai | variate_id ~ {variate_id.shape}")
        ut.print_flush(f"get_dist_moirai | prediction_mask ~ {prediction_mask.shape}")
        gpu_memory_status()
    forecast_model = None
    if not cpu: torch.cuda.empty_cache()
    if verbose > 0:
        ut.print_flush(f"--> get_dist_moirai | Delete Auxiliar model | After Memory:")
        gpu_memory_status()
    
    model_kwargs={
        'target': target, 
        'observed_mask': observed_mask,
        'sample_id': sample_id,
        'time_id': time_id,
        'variate_id': variate_id,
        'prediction_mask': prediction_mask,
        'patch_size': torch.ones_like(sample_id, dtype = torch.float32)*patch_size
    } 
    if verbose > 0: 
        ut.print_flush(f"get_dist_moirai | About to get activations")
    dist = enc_model(**model_kwargs)
    
    
    if not cpu:
        #ut.print_flush(f"get_dist_moirai | enc_input to cpu")
        #enc_input.cpu()
        ut.print_flush(f"get_dist_moirai | enc_model to cpu", verbose = verbose)
        enc_model.cpu()
        ut.print_flush(f"get_dist_moirai | torch cuda empty cache", verbose = verbose)
        torch.cuda.empty_cache()
    if verbose > 0: 
        ut.print_flush("get_dist_moirai -->", verbose = verbose)
    if return_kwargs:
        return dist, model_kwargs
    return dist

In [ ]:
#| export 
def get_enc_embs(
    X               , 
    enc_learn       : Learner, 
    module          : str  = None, 
    cpu             : bool = False, 
    average_seq_dim : bool = True, 
    to_numpy        : bool = True,
    verbose         : int  = 0,
    **kwargs        
):
    embs = None
    enc_learn_class = str(enc_learn.__class__)[8:-2]
    match enc_learn_class:
        case "momentfm.models.moment.MOMENTPipeline":
            match enc_learn.task_name:
                case "embedding":
                    embs = get_enc_embs_moment(X, enc_learn, cpu, to_numpy, verbose, average_seq_dim, **kwargs)
                case "reconstruction":
                    embs = get_enc_embs_moment_reconstruction(X, enc_learn, cpu, to_numpy, verbose, average_seq_dim, **kwargs)
                case _:
                    ut.print_flush(f"Model embeddings for moment-{enc_learn.task_name} is not yet implemented.", verbose = verbose)
        case "fastai.learner.Learner":
            embs = get_enc_embs_MVP_set_stride_set_batch_size(X, enc_learn, stride, batch_size, module, cpu, average_seq_dim, to_numpy, verbose, False, 0, False)
        case "uni2ts.model.moirai.module.MoiraiModule":
            embs = get_enc_embs_moirai(
                enc_input  = X, 
                enc_model  = enc_learn,
                cpu        = cpu, 
                average_seq_dim = average_seq_dim,
                verbose    = verbose,
                **kwargs
            )
        case _:
            ut.print_flush(f"Model embeddings implementation is not yet implemented for {enc_learn_class}.", verbose = verbose)
    return embs

In [ ]:
#| export
def get_enc_embs_set_stride_set_batch_size(
    X                  : List [ List [ List [ float ] ] ], 
    enc_learn          : Learner, 
    stride             : int, 
    batch_size         : int, 
    module             : str  = None, 
    cpu                : bool = False, 
    average_seq_dim    : bool = True, 
    to_numpy           : bool = True, 
    verbose            : int  = 0, 
    time_flag          : bool = False, 
    chunk_size         : int  = 0, 
    check_memory_usage : bool = False,
    **kwargs
):
    ut.print_flush("--> get_enc_embs_set_stride_set_batch_size", verbose = verbose)
    embs = None
    enc_learn_class = str(enc_learn.__class__)[8:-2]
    match enc_learn_class:
        case "momentfm.models.moment.MOMENTPipeline":
            if verbose > 0: 
                ut.print_flush(f"get_enc_embs_set_stride_set_batch_size | Moment | {average_seq_dim}", verbose = verbose)
            match enc_learn.task_name:
                case "embedding":
                    embs = get_enc_embs_moment( X = X, enc_learn = enc_learn, cpu = cpu, to_numpy = to_numpy, verbose = verbose, average_seq_dim = average_seq_dim)
                case "reconstruction":
                    embs = get_enc_embs_moment_reconstruction(X= X, enc_learn = enc_learn, cpu = cpu, to_numpy = to_numpy, verbose = verbose, average_seq_dim = average_seq_dim, **kwargs)
                case _:
                    ut.print_flush(f"Model embeddings for moment-{enc_learn.task_name} is not yet implemented.", verbose = verbose)
        case "fastai.learner.Learner":
            if verbose > 0: 
                ut.print_flush(f"get_enc_embs_set_stride_set_batch_size | MVP | {average_seq_dim}", verbose = verbose)
            if verbose > 1:
                ut.print_flush(f"get_enc_embs_set_stride_set_batch_size | X ~{X.shape}", verbose = verbose)
            embs = get_enc_embs_MVP_set_stride_set_batch_size(
                X = X, 
                enc_learn = enc_learn, 
                stride = stride, 
                batch_size = batch_size, 
                module = module, 
                cpu = cpu, 
                average_seq_dim = average_seq_dim,
                to_numpy = to_numpy, 
                verbose = verbose, 
                time_flag = time_flag, 
                chunk_size = chunk_size, 
                check_memory_usage = check_memory_usage
            )
        case "uni2ts.model.moirai.module.MoiraiModule":
            if verbose > 0: 
                ut.print_flush(f"get_enc_embs_set_stride_set_batch_size | Moirai | {average_seq_dim}", verbose = verbose)
            embs = get_enc_embs_moirai(
                enc_input  = X, 
                enc_model  = enc_learn,
                cpu        = cpu, 
                average_seq_dim = average_seq_dim,
                verbose    = verbose,
                to_numpy = to_numpy,
                **kwargs
            )
        case _:
            ut.print_flush(f"[ get_enc_embs_set_stride_set_batch_size ] Model embeddings implementation is not yet implemented for {enc_learn_class}.", verbose = verbose)
    # Ñapa: TODO: Gestionar que no se queden en memoria los modelos porque ocupan el 40% de la GPU al llamarlos desde R
    if verbose > 0: ut.print_flush(f"get_enc_embs_set_stride_set_batch_size | Before moving to CPU | embs~{embs.shape}", verbose = verbose)
    if cpu:
        #X.cpu()
        enc_learn.cpu()
        try: 
            enc_lear.dls.cpu()
        except Exception as e: 
            ut.print_flush(f"get_enc_embs_set_stride_set_batch_size | Exception: {e}", verbose = verbose)
        #kwargs_to_cpu_(**kwargs)
    if verbose > 0: ut.print_flush(f"get_enc_embs_set_stride_set_batch_size | embs~{embs.shape} -->", verbose = verbose)
    return embs

## Validation with customized metrics

### Customized metrics

In [ ]:
#| export
def rmse(preds, targets):
    res = torch.sqrt(torch.nn.functional.mse_loss(preds, targets))
    return res

def smape(preds, targets):
    res = 100 * torch.mean(2 * torch.abs(preds - targets) / (torch.abs(preds) + torch.abs(targets)))
    return res

def rmse_flat(preds, targets):
    """
    Computes RMSE while flattening the tensors to ensure compatibility with MSELossFlat.
    """
    preds, targets = preds.view(-1), targets.view(-1)  # Flatten tensors
    return torch.sqrt(torch.nn.functional.mse_loss(preds, targets))

def smape_flat(preds, targets):
    """
    Computes SMAPE while flattening the tensors to ensure compatibility with MSELossFlat.
    """
    preds, targets = preds.view(-1), targets.view(-1)  # Flatten tensors
    denominator = (torch.abs(preds) + torch.abs(targets))
    return 100 * torch.mean(2 * torch.abs(preds - targets) / torch.clamp(denominator, min=1e-7))
def mae_flat(preds, targets):
    """
    Computes Mean Absolute Error (MAE) while flattening the tensors to ensure compatibility.
    """
    preds, targets = preds.view(-1), targets.view(-1)  # Flatten tensors
    return torch.mean(torch.abs(preds - targets))

def mse_loss_flat(preds, targets):
    """
    Computes Mean Squared Error (MSE) while flattening the tensors to ensure compatibility.
    """
    preds, targets = preds.view(-1), targets.view(-1)  # Flatten tensors
    return torch.mean((preds - targets) ** 2)

In [ ]:
#| export
class RMSELoss(_Loss):
    __constants__ = ["reduction"]
    def __init__(self, size_average = None, reduce = None, reduction: str = "mean") -> None:
        super().__init__(size_average, reduce, reduction)
    
    def forward(self, input: Tensor, target:Tensor) -> Tensor:
        return torch.nn.functional.mse_loss(input, target, reduction = self.reduction)

@use_kwargs_dict(reduction='mean')
def RMSELossFlat(
    *args,
    axis:int = -1,
    floatify: bool = True, 
    **kwargs
):
    "Computes RMSE with flattening, similar to MSELossFlat."
    return BaseLoss(RMSELoss, *args, axis = axis, floatify = floatify, is_2d = False, **kwargs)

class SMAPELoss(_Loss):
    __constants__ = ["reduction"]
    
    def __init__(self, size_average=None, reduce=None, reduction: str = "mean") -> None:
        """
        Initializes the SMAPE Loss.
        
        Args:
            size_average (bool, optional): Deprecated (use reduction).
            reduce (bool, optional): Deprecated (use reduction).
            reduction (str): Specifies the reduction to apply to the output ('none', 'mean', 'sum').
        """
        super().__init__(size_average, reduce, reduction)
    
    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        """
        Computes the SMAPE loss.
        
        Args:
            input (Tensor): Predicted values.
            target (Tensor): Ground truth values.
        
        Returns:
            Tensor: Computed SMAPE loss.
        """
        return self.smape_loss(input, target)
    
    @staticmethod
    def smape_loss(preds: Tensor, targets: Tensor) -> Tensor:
        """
        Computes the SMAPE loss for the given predictions and targets.
        
        Args:
            preds (Tensor): Predicted values.
            targets (Tensor): Ground truth values.
        
        Returns:
            Tensor: SMAPE loss.
        """
        denominator = (torch.abs(preds) + torch.abs(targets))
        smape = 100 * torch.mean(2 * torch.abs(preds - targets) / torch.clamp(denominator, min=1e-7))
        return smape


@use_kwargs_dict(reduction="mean")
def SMAPELossFlat(
    *args,
    axis: int = -1,
    floatify: bool = True,
    **kwargs
):
    """
    Computes SMAPE with flattening, similar to MSELossFlat.
    
    Args:
        axis (int): Axis to flatten. Default is -1.
        floatify (bool): Convert target to float. Default is True.
        **kwargs: Additional arguments.
    """
    return BaseLoss(SMAPELoss, *args, axis=axis, floatify=floatify, is_2d=False, **kwargs)

# Class alias for clarity
MAELossFlat = L1LossFlat

# Evaluation metrics alias for clarity
try:
    EvalMSE     = evaluate.load('mse', 'multilist')
    EvalRMSE    = evaluate.load('mse', 'multilist')
    EvalMAE     = evaluate.load('mae', 'multilist')
    EvalSMAPE   = evaluate.load('smape', 'multilist')
except:
    try:
        metrics_dir = os.path.join("~/work/nbs/evaluate_metrics")
        EvalMSE   = evaluate.load(metrics_dir+"/mse")
        EvalRMSE  = evaluate.load(metrics_dir+"/mse")
        EvalMAE   = evaluate.load(metrics_dir+"/mae")
        EvalSMAPE = evaluate.load(metrics_dir+"/smape")
    except: # Copied from evaluate 
    #(Ñapa por problemas de conexión con HF / no detección de carpeta. TODO: Arreglar)
        from dvats.evaluate_metrics import mae, mse, smape
        EvalMSE     = mse.Mse(config_name = 'multilist')
        EvalRMSE    = mse.Mse(config_name = 'multilist')
        EvalMAE     = mae.Mae(config_name = 'multilist')
        EvalSMAPE   = smape.Smape(config_name = 'multilist')

### Computation & structure

In [ ]:
#| export
def validate_with_metrics(
    learner : Learner, 
    metrics        = None,
    default_metric = MSELossFlat #- Default for MVP
):
    """
    Validates a `Learner` using multiple metrics and restores the default metric after computation.

    Args:
        learner (Learner): The `Learner` object to validate.
        metrics (list, optional): A list of metric functions to use for validation. 
            If not provided or empty, the `default_metric` will be used. 
            Each metric function should be compatible with the `Learner`'s `crit` attribute.
        default_metric (Callable, optional): The default metric to restore after validation. 
            Defaults to `MSELossFlat`.

    Returns:
        list: A list of validation results, where each result corresponds to the validation 
        score for the respective metric in the `metrics` list.

    Raises:
        Warning: If no metric is provided, a warning is issued, and the `default_metric` is used.

    Example:
        >>> learner = Learner(...)  # Initialize your learner
        >>> metrics = [MSELossFlat, L1Loss]
        >>> results = validate_with_metrics(learner, metrics)
        >>> print(results)  # [0.25, 0.35]

    Notes:
        - The function temporarily replaces the `crit` attribute of the `Learner` object 
          with each metric in the `metrics` list for validation.
        - After validation, the `crit` attribute is restored to the `default_metric`.
    """
    metrics_dict = {}
    #-- Ensure well formated metrics list
    if metrics is None or len(metrics) == 0:
        warnings.warn(f"No metric selected. Using default: {MSELossFlat.__name__}")
        metrics = [MSELossFlat]
        metrics_dict = {
            'mse': {
                'metric': MSELossFlat,
                'args': {}
            }
        }
    #-- Compute the results by
    # 1) Changing learner criteria to the current metric
    # 2) Use learner.validate()
    results = []
    for metric in metrics:
        learner.crit = metric
        result = learner.validate()
        result = result.item() if hasattr(result, 'item') else result
        results.append(result)
    
    #-- Restore default metric
    learner.crit=default_metric
    return results, metrics_dict

def validate_with_metrics_(
    self : Encoder
): 
    results, metrics_dict = validate_with_metrics(
        learner        = self.model, 
        metrics        = self.metrics, 
        default_metric = self.default_metric
    )
    self.metrics_dict = self.metrics_dict if self.metrics_dict else metrics_dict
    return results

Encoder.validate_with_metrics_ = validate_with_metrics_

In [ ]:
#| export
def get_metrics_dict(
    metrics: List[Callable],
    metrics_names: List[str] = None
) -> Dict[str, Callable]:
    """
    Creates a dictionary mapping metric names to metric functions.

    Args:
        metrics (List[Callable]): A list of metric functions.
        metrics_names (List[str], optional): A list of metric names corresponding to the functions.

    Returns:
        Dict[str, Callable]: A dictionary mapping metric names to their corresponding functions.
    """
    if metrics_names is None:
        # Use the function's name as the key if names are not provided
        metrics_dict = {metric.__name__: metric for metric in metrics}
    elif len(metrics) == len(metrics_names):
        # Use provided names if lengths match
        metrics_dict = {name: metric for name, metric in zip(metrics_names, metrics)}
    else:
        raise ValueError("Length of metrics and metrics_names must match when metrics_names is provided.")
    return metrics_dict

In [ ]:
#| export
def validate_with_metrics_format_results(
    results, metrics, metrics_names = None
):
    if metrics_names is None:
        # Use the function's name as the key if names are not provided
        metrics_dict = {
            metric.__name__: results[i] for 
            (i, metric) in enumerate(metrics)
        }
    elif len(metrics) == len(metrics_names):
        # Use provided names if lengths match
        metrics_dict = {
            name: results[i] for 
            name, (i, metric) in 
            zip(metrics_names, enumerate(metrics))
        }
    else:
        raise ValueError("Length of metrics and metrics_names must match when metrics_names is provided.")
    return metrics_dict

In [ ]:
#| hide
foo = validate_with_metrics_format_results(
    [3,1,2,9],
    metrics_names = ["a", "b", "c", "d"],
    metrics = [RMSELossFlat, MSELossFlat, MAELossFlat, SMAPELossFlat]
)
print(foo)
foo = validate_with_metrics_format_results(
    [3,1,2,9],
    [RMSELossFlat, MSELossFlat, MAELossFlat, SMAPELossFlat],
    None
)
print(foo)

## Fine-tunning
> Take a look on [HuggingFace - Fine-tune a pretrained model](https://huggingface.co/docs/transformers/training) if not used to few-shot learning or fine-tuning models.

Steps: 

1) Prepare the dataset
2) Batch the data
   - Remember splitting between train & test dataset
   - Remember to use DataLoader to iterate over batches
4) Load the trained model and check if any modification is needed
   - Check wether any layer may be substituted by an "identity" if not needed for your case
   - Check if any dimension in a conversion layer may be changed to fit your dataset.
5) Select an optimizer from torch.optim (Adam)
6) ¿If using transformer, lr_scheduler? 
7) Training loop
     

### Utils

In [ ]:
#| export
def random_windows(
    X           : List[List[List[float]]], 
    n_windows   : int       = None, 
    percent     : float     = None, 
    mssg        : ut.Mssg   = ut.Mssg()
):
    """
    Parameters: 
    - X: Windows array. Expected shape: [batch_size or n_samples, n_vars, window_len]
    
    Given a window array, selects:
    - n_windows random windows from the array if n_windows is provided.
    - ceil(percent * len(X)) random windows otherwise.
    
    Indices are selected without repetition and are sorted in ascending order.
    """
    # Setup mssg
    func = mssg.function
    mssg.level += 1
    mssg.initial(ut.funcname())
    
    mssg.print(f"N windows: {n_windows}")
    
    if n_windows is None and percent is None:
        # Retrieve everything as a tensor
        if isinstance(X, torch.Tensor):  # Ensure correct type checking
            windows = X.float()
        else:
            windows = torch.from_numpy(X).float()
    else: 
        if n_windows is None:
            if percent is None:
                raise ValueError("Either n_windows or percent must be provided as a float value.")
            n_windows = int(np.ceil(percent * X.shape[0]))
        else: 
            n_windows = int(min(X.shape[0], n_windows))
        
        mssg.print_error(f"n_windows: {n_windows}")
        if n_windows > X.shape[0]:
            warnings.warn(f"n_windows {n_windows} > {X.shape[0]} (total number of windows). Giving all.")
            n_windows = X.shape[0]
            random_indices = np.arange(n_windows)
        else:
            # Get unique random indices
            random_indices = np.random.choice(  
                X.shape[0], 
                size=n_windows, 
                replace=False
            )
            # Sort indices in ascending order
            random_indices = np.sort(random_indices)
        mssg.print_error(f"Index (smallest): {random_indices[0]}")
        mssg.print_error(f"Index (largest): {random_indices[-1]}")
        windows = X[random_indices]
        # Ensure torch tensor
        if isinstance(windows, torch.Tensor):
            windows = windows.float()
        else:
            windows = torch.from_numpy(windows).float()
    
    mssg.print(f"windows~{windows.shape}")
    mssg.final()
    
    # Restore mssg
    mssg.level      -= 1
    mssg.function   = func
    
    return windows, random_indices


In [ ]:
#| export
def windowed_dataset(
    X                               : Union [ List [ List [ List [ float ]]], List [ float ], pd.DataFrame ],
    stride                          : int           = 1,
    window_sizes                    : List [int]    = None,
    n_window_sizes                  : int           = 1,
    window_sizes_offset             : int           = 0.05,
    windows_min_distance            : int           = 1,
    full_dataset                    : bool          = False,
    mix_windows                     : bool          = False,
    validation_percent              : float         = 0.3,
    training_percent                : float         = 1.0,
    cpu                             : bool          = False,
    mssg                            : ut.Mssg       = ut.Mssg()
): 
    stride = 1 if stride is None else stride 
    n_window_sizes = 1 if n_window_sizes is None else n_window_sizes
    window_sizes_offset = 0.05 if window_sizes_offset is None else window_sizes_offset
    windows_min_distance = 1 if windows_min_distance is None else windows_min_distance
    full_dataset = False if full_dataset is None else full_dataset
    mssg = ut.Mssg() if mssg is None else mssg
    mssg.level += 1
    mssg.initial(ut.funcname())
    dss = []
    if window_sizes is None or n_window_sizes > len(window_sizes):
        mssg.print("X is a DataFrame | Selecting Fourier's dominant frequences")
        mssg.level += 1
        # Select Fourier's dominant frequences
        window_sizes_ = find_dominant_window_sizes_list(
            X               = X, 
            nsizes          = n_window_sizes, 
            offset          = window_sizes_offset, 
            min_distance    = windows_min_distance,
            mssg            = mssg
        )
        mssg.level -= 1
        window_sizes = window_sizes_ if window_sizes is None else list(set(window_sizes + window_sizes_))[:n_window_sizes]
    if mix_windows:
        dss = ut.WindowedDataset(
            dataset             = X,
            window_sizes        = window_sizes,
            validation_percent  = validation_percent,
            training_percent    = training_percent,
            device              = "cpu" if cpu else torch.cuda.current_device(),
            stride              = stride
        )
    if isinstance(X, list):
        mssg.print("X is a list. Converting to dataFrame")
        X = np.array(X)
        X = pd.DataFrame(X)        
    if ( isinstance(X,pd.DataFrame) or full_dataset): 
        mssg.print(f"X is a DataFrame, X~{X.shape} | window_sizes {len(window_sizes) if window_sizes is not None else 0}, n_window_sizes {n_window_sizes}")
        mssg.print(f"X is a DataFrame | Window sizes: {len(window_sizes)}", func_name = ut.funcname())
        mssg.print(f"Building the windows")
        if not mix_windows:
            for w in window_sizes:
                mssg.print(f"w = {w}", verbose_level = mssg.level+1)
                enc_input, _ = SlidingWindow(window_len = w, stride = stride, get_y=[])(X)
                dss.append(enc_input)
                mssg.print(f"w {w} | enc_input~{enc_input.shape} | dss~{len(dss)}",  verbose_level = mssg.level+1, func_name = ut.funcname())
    else: 
        mssg.print("X is already windowed")
        dss = [X]
    if not mix_windows: mssg.print(f"Number of windows: {len(dss)}")
    mssg.final()
    mssg.level -= 1
    return dss, window_sizes


In [ ]:
#| export
def setup_scheduler(
    self : Encoder,
    dl_train,
    optimizer                             = None,
    num_training_steps              : int = None,
    lr_scheduler_perc_warmup_steps  : int = 0.02,
    lr_scheduler_max_lr             : float = None,
):
    num_training_steps = num_epochs * len(dl_train) if num_training_steps is None else num_training_steps
    if not self.optim.lr.num_warmup_steps:
        lr_scheduler_num_warmup_steps = lr_scheduler_perc_warmup_steps*num_training_steps
    lr_scheduler_max_lr = 5 - 10 *self.optim.lr.lr if lr_scheduler_max_lr is None else lr_scheduler_max_lr
    if self.optim.lr.flag:
        match self.optim.lr.name:
            case "OneCycleLR": 
                lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer           = optimizer,
                    max_lr              = lr_scheduler_max_lr,
                    epochs              = self.num_epochs,
                    steps_per_epoch     = len(dl_train)
                )
            case _:
                lr_scheduler = get_scheduler(
                    name                = self.optim.lr.name,
                    optimizer           = optimizer,
                    num_warmup_steps    = lr_scheduler_num_warmup_steps,
                    num_training_steps  = num_training_steps
                )
    return lr_scheduler
Encoder.setup_scheduler = setup_scheduler

In [ ]:
#| export
def set_train_size(
    self        : Encoder,
    sample_id   : int,
    eval_size   : int
):
    # Compute sizes / percent
    total_size  = self.input.data[sample_id].shape[0]
    if self.input.validation_percent > 0.3:
        raise ValueError("Validation percent is too big. Please give a value lower or equal than 0.3.")
    if self.input.validation_percent is None: 
        self.input.validation_percent = 0.3
    if (
        self.input.n_windows is None 
        and self.input.n_windows_percent is None
    ):
        train_size  = np.ceil(self.input.training_percent * (total_size-eval_size))
    else:
        if self.input.n_windows is None:
            total_windows = total_size*self.input.n_windows_percent
        else:
            total_windows = self.input.n_windows
        train_size  = np.ceil(total_windows-eval_size)*self.input.training_percent
    return int(train_size)
Encoder.set_train_size = set_train_size

In [ ]:
#| export
def set_train_and_eval_sizes(
    self        : Encoder,
    sample_id   : int
):
    # Compute sizes / percent
    total_size  = self.input.data[sample_id].shape[0]
    if self.input.validation_percent > 0.3:
        raise ValueError("Validation percent is too big. Please give a value lower or equal than 0.3.")
    if self.input.validation_percent is None: 
        self.input.validation_percent = 0.3
    if (
        self.input.n_windows is None 
        and self.input.n_windows_percent is None
    ):
        eval_size   = np.ceil(self.input.validation_percent * total_size)
        train_size  = np.ceil(self.input.training_percent * (total_size-eval_size))
    else:
        if self.input.n_windows is None:
            total_windows = total_size*self.input.n_windows_percent
        else:
            total_windows = self.input.n_windows
        eval_size   = np.ceil(total_windows*self.input.validation_percent)
        train_size  = np.ceil(total_windows-eval_size)*self.input.training_percent
    return int(eval_size), int(train_size)
Encoder.set_train_and_eval_sizes = set_train_and_eval_sizes

In [ ]:
#| export
def set_random_dataset(
    self        : Encoder,
    rand_size   : int,
    dataset
):  
    ds_rand = None
    ds_rand, rand_indices = random_windows(
        X           = dataset,
        n_windows   = rand_size,
        percent     = None, # Unneccesary
        mssg        = self.mssg
    )
    self.mssg.print_error(f"Total: {dataset.shape[0]}")
    self.mssg.print_error(f"Eval: {rand_size} | {ds_rand.shape[0]}")
    return ds_rand, rand_indices
Encoder.set_random_dataset = set_random_dataset

In [ ]:
#| export
def set_train_dataset(
    self,
    ds,
    dl_eval_indices,
    train_size
):
    self.mssg.print(f"Train DataLoader | Random windows")
    mask                        = torch.ones(ds.shape[0], dtype = bool)
    mask[dl_eval_indices]       = False
    ds_diff                     = ds[mask]
    ds_train, train_indices  = self.set_random_dataset(
        train_size, ds_diff
    )
    return ds_train, train_indices
Encoder.set_train_dataset = set_train_dataset

In [ ]:
#| export
def set_train_and_eval_dataloaders(
    self                : Encoder,
    sample_id           : int
):
    # Setup mssg
    func = self.mssg.function
    self.mssg.level += 1
    self.mssg.initial_(ut.funcname())
    _case_ = (
        # Ensure that you always get the same validation set
        # no matters training or batch sizes
        self.ws, 
        self.input.validation_percent,
        self.input.n_windows,
        self.input.n_windows_percent
    )
    ds = self.input.data[sample_id]
    # Go!
    if self.eval_indices_dict is None: self.eval_indices_dict = {}
    if _case_ in self.eval_indices_dict:
        self.mssg.print_error(f"Use cached indices for case = {_case_}")
        eval_indices = self.eval_indices_dict[_case_]
        self.mssg.print_error(f"Eval indices: {eval_indices[0]}-{eval_indices[1]}")
        ds_eval             = ds[eval_indices]
        eval_size           = ds_eval.shape[0]
        train_size          = self.set_train_size(sample_id, eval_size)
        ds_train, _         = self.set_train_dataset(
            ds              = ds,
            dl_eval_indices = eval_indices, 
            train_size      = train_size
        )
    else:
        self.mssg.print(f"Computing indices for case = {_case_}")
        eval_size, train_size = self.set_train_and_eval_sizes(sample_id)
        ds = torch.from_numpy(self.input.data[sample_id]).float()
        self.mssg.print(f"Selecting validation dataset | windows")
        ds_eval, eval_indices = self.set_random_dataset(eval_size, ds)
        self.eval_indices_dict[_case_] = eval_indices
        self.mssg.print(f"Saved indices {eval_indices} for case {_case_}")
        ds_train, _ = self.set_train_dataset(
            ds              = ds,
            dl_eval_indices = eval_indices, 
            train_size      = train_size
        )
        self.mssg.print(f"Saved indices after train dataset {eval_indices} for case {_case_}")
    dl_eval  = DataLoader(
        dataset     = ds_eval, 
        batch_size  = self.input.batch_size, 
        shuffle     = False
    )
    dl_train  = DataLoader(
        dataset     = ds_train, 
        batch_size  = self.input.batch_size, 
        shuffle     = False
    )
    self.mssg.final()
    # Restore mssg
    self.mssg.level -= 1
    self.mssg.function = func
    return dl_eval, dl_train, ds_eval, ds_train
Encoder.set_train_and_eval_dataloaders = set_train_and_eval_dataloaders

In [ ]:
#| export
def _set_enc_input(
    mssg                            : ut.Mssg,
    # Encoder Input
    ## -- Using all parammeters
    X                               : Optional [ 
                                        Union [ 
                                            List [ List [ List [ float ]]], 
                                            List [ float ], 
                                            pd.DataFrame 
                                        ] 
                                    ],
    stride                          : Optional [ int ]          = None,
    batch_size                      : Optional [ int ]          = None,
    n_windows                       : Optional [ int ]          = None,
    n_windows_percent               : Optional [ float ]        = None,
    validation_percent              : Optional [ float ]        = None,
    training_percent                : Optional [ float ]        = None,
    window_mask_percent             : Optional [ float ]        = None,
    window_sizes                    : Optional [ List [int] ]   = None,
    n_window_sizes                  : Optional [ int ]          = 1,
    window_sizes_offset             : Optional [ int ]          = 0.05,
    windows_min_distance            : Optional [ int ]          = 1,
    full_dataset                    : Optional [ bool ]         = False,
    ## -- Using Type
    enc_input                       : Optional [ EncoderInput ] = None,
    mix_windows                     : bool                      = False,
    cpu                             : bool                      = False
): 
    # Setup mssg
    mssg.level += 1
    func = mssg.function
    mssg.initial_(func_name = ut.funcname())
    # Go!
    enc_input, _ = ut._check_value(enc_input, None, "enc_input", EncoderInput, True, False, False)
    mssg.print(f"is none enc_input? {enc_input is None}")
    if enc_input is None:
        mssg.print(f"About to get the windows")
        enc_input, window_sizes = windowed_dataset(
            X                       = X,
            stride                  = stride,
            window_sizes            = window_sizes,
            n_window_sizes          = n_window_sizes,
            window_sizes_offset     = window_sizes_offset,
            windows_min_distance    = windows_min_distance,
            full_dataset            = full_dataset,
            mix_windows             = mix_windows,
            validation_percent      = validation_percent,
            training_percent        = training_percent,
            cpu                     = cpu,
            mssg                    = mssg
        )
        mssg.print(f"About to get the encoder input", func_name = ut.funcname())
        enc_input = EncoderInput(
            _data               = enc_input, 
            stride              = stride,
            batch_size          = batch_size,
            n_windows           = n_windows,
            n_windows_percent   = n_windows_percent,
            validation_percent  = validation_percent,
            training_percent    = training_percent,
            window_mask_percent = window_mask_percent
        )
        if not mix_windows: mssg.print(f"Enc input obtained | enc_input~{enc_input.shape}")
    mssg.final()
    # Restore mssg
    mssg.function = func
    mssg.level -= 1
    return enc_input, window_sizes

In [ ]:
#| export
def _set_optimizer(
    mssg                            : ut.Mssg,
    optim                           : EncoderOptimizer = None,
    criterion                       : _Loss         = torch.nn.MSELoss, 
    optimizer                                       = None, 
    lr                              : float         = 5e-5, #1e-4, 
    lr_scheduler_flag               : bool          = False, 
    lr_scheduler_name               : str           = "linear",
    lr_scheduler_num_warmup_steps   : int           = None
):
    # Setup mssg
    func = mssg.function
    mssg.level += 1
    mssg.initial(ut.funcname())
    # Go!
    optim,_ = ut._check_value(optim, None, "optim", EncoderOptimizer, True)
    if optim is None:
        optim = EncoderOptimizer(
            criterion   = criterion,
            optimizer   = optimizer,
            lr          = LRScheduler (
                            lr              = lr,
                            flag            = lr_scheduler_flag,
                            name            = lr_scheduler_name,
                            num_warmup_steps= lr_scheduler_num_warmup_steps
            ),
        )
    mssg.final()
    # Restore mssg 
    mssg.function = func 
    mssg.level -= 1
    return optim

In [ ]:
#| export
def _set_encoder(
    ## -- Using all parammeters
    X                               : Optional [ Union [ List [ List [ List [ float ]]], List [ float ], pd.DataFrame ] ],
    stride                          : Optional [ int ]          = None,
    batch_size                      : Optional [ int ]          = None,
    n_windows                       : Optional [ int ]          = None,
    n_windows_percent               : Optional [ float ]        = None,
    validation_percent              : Optional [ float ]        = None, 
    training_percent                : Optional [ float ]        = None,
    window_mask_percent             : Optional [ float ]        = None,
    window_sizes                    : Optional [ List [int] ]   = None,
    n_window_sizes                  : Optional [ int ]          = 1,
    window_sizes_offset             : Optional [ int ]          = 0.05,
    windows_min_distance            : Optional [ int ]          = 1,
    full_dataset                    : Optional [ bool ]         = False,
    ##-- Given by Type 
    enc_input                       : Optional [ EncoderInput ] = None,
    # Optimizer
    optim                           : Optional [ EncoderOptimizer ] = None,
    ## -- Using all parameters
    criterion                       : Optional [ _Loss ]            = torch.nn.MSELoss, 
    optimizer                                                       = None, 
    lr                              : Optional [ float ]            = 5e-5, #1e-4, 
    lr_scheduler_flag               : Optional [ bool ]             = False, 
    lr_scheduler_name               : Optional [ str ]              = "linear",
    lr_scheduler_num_warmup_steps   : Optional [ int ]              = None,
    # Mssg
    ## -- Using all parameters
    verbose                         : Optional[ int ]               = 0, 
    print_to_path                   : Optional[ bool ]              = False,
    print_both                      : Optional[bool]                = False,
    print_path                      : Optional[ str ]               = "~/data/logs/logs.txt",
    print_mode                      : Optional[ str ]               = 'a',
    ## -- Using Type
    mssg                            : Optional [ ut.Mssg ]          = None,
    ## Encoder 
    enc                             : Optional [ Encoder ]          = None,
    ## -- Using all parameters
    num_epochs                      : Optional [ int]               = 3,
    enc_learn                       : Optional [Learner]            = None, 
    cpu                             : Optional [ bool ]             = False,
    to_numpy                        : Optional [ bool ]             = True,
    #- Masking options
    mask_stateful                   : Optional [ bool ]             = False,
    mask_future                     : Optional [ bool ]             = False,
    mask_sync                       : Optional [ bool ]             = False,
    #- Loss criterions
    
    metrics                         : Optional [ List [ Callable ]] = None,
    metrics_names                   : Optional [ List [ str ] ]     = None,
    metrics_args                    : Optional [ AttrDict ]         = None,
    metrics_dict                    : Optional [ AttrDict ]         = None,
    scheduler_specific_kwargs       : Optional [ AttrDict ]         = None,
    mix_windows                     : Optional [ bool ]             = False
):
    enc,_ = ut._check_value(enc, None, "enc", Encoder, True)
    
    if enc is None: 
        mssg = ut._set_mssg(
            mssg          = mssg,
            verbose       = verbose, 
            print_to_path = print_to_path, 
            print_path    = print_path, 
            print_mode    = print_mode,
        )
        mssg.both = print_both
        mssg.initial(ut.funcname())
        mssg.print("About to exec _set_enc_input")
        enc_input, window_sizes = _set_enc_input(
            mssg                = mssg, 
            X                   = X, 
            stride              = stride, 
            batch_size          = batch_size, 
            n_windows           = n_windows, 
            n_windows_percent   = n_windows_percent, 
            validation_percent  = validation_percent, 
            training_percent    = training_percent, 
            window_mask_percent = window_mask_percent, 
            window_sizes        = window_sizes, 
            n_window_sizes      = n_window_sizes, 
            window_sizes_offset = window_sizes_offset, 
            windows_min_distance= windows_min_distance, 
            full_dataset        = full_dataset,
            enc_input           = enc_input, 
            mix_windows         = mix_windows,
            cpu                 = cpu
        )
        mssg.print(f"enc_input~{enc_input.shape}")
        mssg.print("About to exec _set_optimizer")
        optim = _set_optimizer(mssg, optim, criterion, optimizer, lr, lr_scheduler_flag, lr_scheduler_name, lr_scheduler_num_warmup_steps)
        
        if metrics_dict is None and (
                metrics and metrics_names and metrics_args and 
                len(metrics) == len(metrics_names) == len(metrics_args)
            ):
            metrics_dict = {
                metrics_names[i]: {
                    'metric': metrics[i],
                    'args'  : metrics_args[i]
                }
                for i in range(len(metrics_names))
            }
            
        enc = Encoder(
            model                       = enc_learn,
            input                       = enc_input,
            mssg                        = mssg,
            cpu                         = cpu,
            to_numpy                    = to_numpy, 
            num_epochs                  = num_epochs, 
            optim                       = optim,
            mask_stateful               = mask_stateful,
            mask_future                 = mask_future,
            mask_sync                   = mask_sync,
            eval_stats_pre              = None,
            eval_stats_post             = None,
            metrics_dict                = metrics_dict,
            scheduler_specific_kwargs   = scheduler_specific_kwargs
        )
        if mix_windows:
            enc.window_sizes = window_sizes

    enc.mssg.final(ut.funcname())
    return enc

### Moment
> Follow the tutorial in the original repository: [Moment - Imputation](https://github.com/moment-timeseries-foundation-model/moment/blob/main/tutorials/imputation.ipynb).

In [ ]:
#| export
def fine_tune_moment_compute_loss_check_sizes_(
    batch           : List [ List [ List [ float ] ] ], 
    output,
    mssg
):
    # setup mssg
    func = mssg.function
    mssg.level += 1
    mssg.initial_(ut.funcname())
    #b = batch.clone()
    B    = batch
    b_2  = batch.shape[2]
    re_2 = output.reconstruction.shape[2]
    
    if b_2 > re_2:
        mssg.print(f" TODO: Why? Original {b_2} > {re_2}  Reconstruction")
        b = b[...,:re_2]
    elif re_2 > b_2:
        mssg.print(f" Why ? Original {b_2} < {re_2} Reconstruction ? Padding")
        output.reconstruction = output.reconstruction[...,:b_2]
    else: 
        mssg.print(f" re_2 {re_2} == {b_2} y_2")
    mssg.print(f"---------- Checking loss  ------- | reconstruction ~ {output.reconstruction.shape} | original_ ~ {b.shape}")
    # Restore mssg
    mssg.level -= 1
    mssg.final()
    mssg.function = func
    return b

In [ ]:
#| export
def moment_compute_loss(
    self        : Encoder,
    batch, 
    output, 
    input_mask  = None, 
    mask        = None
):
    #Configure mssg
    func = self.mssg.function 
    self.mssg.level += 1
    self.mssg.initial_(ut.funcname())
    #Check sizes
    #references = fine_tune_moment_compute_loss_check_sizes_(
    #    batch   = batch, 
    #    output  = output, 
    #    mssg    = self.mssg
    #)
    references  = batch
    predictions = output.reconstruction
    self.mssg.print(f"b~{references.shape} | o~{predictions.shape}")
    # Get Masks
    batch_masks = output.input_mask if input_mask is None else input_mask
    mask        = output.pretrain_mask if mask is None else mask
    # Move tensors to device
    device      = predictions.device if references.device != "cpu" else references.device
    references  = references.to(device)
    predictions = predictions.to(device)
    batch_masks = batch_masks.to(device)
    mask        = mask.to(device)
    self.mssg.print(f"batch ~ {batch.shape} | {batch.device}")
    self.mssg.print(f"batch_masks ~ {batch_masks.shape} | {batch_masks.device}")
    self.mssg.print(f"mask ~ {mask.shape} | {mask.device}")
    # Compute loss
    self.mssg.print(f"Criterion: {self.optim.criterion}")
    # Ensure sizes
    min_len = min(predictions.shape[2], references.shape[2])
    references  = references[:,:,:min_len]
    predictions = predictions[:,:,:min_len]
    ####
    recon_loss      = self.optim.criterion(predictions, references)
    self.mssg.level += 1
    self.mssg.print(f"Reconstruction loss: {recon_loss}")
    self.mssg.print(f"Batch mask: {batch_masks}")
    self.mssg.print(f"Mask: {mask}")
    observed_mask   = batch_masks * (1-mask)
    self.mssg.print(f"Observed mask: {observed_mask}")
    masked_loss     = observed_mask * recon_loss
    loss            = masked_loss.nansum() / (observed_mask.nansum() + 1e-7)
    self.mssg.print(f"Loss type: {type(loss)}")
    self.mssg.level -= 1
    self.mssg.print(f"loss: {loss.item()}")
    self.mssg.final()
    # Restore mssg
    self.mssg.function = func 
    self.mssg.level -= 1
    return loss

Encoder.moment_compute_loss = moment_compute_loss

In [ ]:
#| export
def fine_tune_moment_eval_preprocess(
    self        : Encoder,
    predictions : List [ List [ float ]],
    references  : List [ List [ float ]],
):
    """
    Parameters:
    - predictions torch (float)
    - references torch (float)
    Returns: 
        - Predictions and references ensuring same shape and no NaN values. 
        - Uses the shape of the smallest torch for the modification.
    """
    #-- Mssg verbose & name
    func = self.mssg.function
    self.mssg.level += 1
    self.mssg.initial_(ut.funcname())
    #--- Go!
    self.mssg.print(f"Before reshape | preds~{predictions.shape}")
    self.mssg.print(f"Before reshape | refs~{references.shape}")
    # Valid shape
    predictions = einops.rearrange(predictions, "b v w -> (b v) w")
    references  = einops.rearrange(references, "b v w -> (b v) w")
    # Common shape 
    if predictions.shape[1] > references.shape[1]: predictions = predictions[:,:references.shape[1]]
    if predictions.shape[1] < references.shape[1]: references = references[:,:predictions.shape[1]]
    self.mssg.print(f"After reshape | preds~{predictions.shape}")
    self.mssg.print(f"After reshape | refs~{references.shape}")
    # Avoid Nans    
    nan_mask    = torch.isnan(predictions) | torch.isnan(references)
    predictions = torch.where(nan_mask, torch.tensor(0.0), predictions)
    references  = torch.where(nan_mask, torch.tensor(0.0), references)
    self.mssg.print(f"After NaN | preds~{predictions.shape}")
    self.mssg.print(f"After NaN | refs~{references.shape}")
    self.mssg.final()
    #-- Restore mssg
    self.mssg.level -= 1
    self.mssg.function = func
    return predictions, references

Encoder.fine_tune_moment_eval_preprocess = fine_tune_moment_eval_preprocess

In [ ]:
#| export 
def get_mask_moment(
    batch, batch_masks, r, mssg
):
    """
    Generates a mask using the MOMENT-style masking approach.
    
    Parameters:
    - batch: Input tensor.
    - batch_masks: Existing batch masks.
    - r: Mask ratio (percentage of masked values).
    - mssg: Message handler for logging.
    
    Returns:
    - mask: Generated mask.
    """
    # Setup mssg
    func = mssg.function
    mssg.level += 1
    mssg.initial(ut.funcname())
    # Go! 
    mssg.print(f"Using mask generator with mask ratio {r}")
    # Create mask generator with percentage of unknown values 'r'
    mask_generator = Masking(
        mask_ratio  = r,
        patch_len   = 1 #Tratando de enmascarar a nivel de elementos.
    )
    # Generate mask
    # [batch size x n_features x window_sizes] -> [batch_size x window_size]
    # Randomness is given at mask_generator.patch_len blocks (no matter the dimension)
    mask = mask_generator.generate_mask(
        x           = batch,
        input_mask  = batch_masks
    )
    mssg.print(f"Mask~{mask.shape}")
    mssg.final()
    # Restore mssg
    mssg.function = func 
    mssg.level -= 1
    return mask

def get_mask_tsai(
    batch, batch_masks, r, mask_stateful, mask_future, mask_sync, mssg
):
    """
    Generates a mask using the TSAI-style masking approach.
    
    Parameters:
    - batch: Input tensor.
    - batch_masks: Existing batch masks.
    - r: Mask ratio (percentage of masked values).
    - mask_stateful: Whether to use a stateful mask.
    - mask_future: Whether to mask future values.
    - mask_sync: Whether to synchronise masking across samples.
    - mssg: Message handler for logging.
    
    Returns:
    - mask: Generated mask.
    """
    # Setup mssg
    func = mssg.function
    mssg.level += 1
    mssg.initial(ut.funcname())
    # Go! 
    mssg.print("Using MVP masking generation style")
    o = torch.zeros(batch.shape[0], batch.shape[2])
    mssg.print(f"o ~ {o.shape} | stateful = {mask_stateful} | sync = {mask_sync} | r = {window_mask_percent}")
    if mask_future:
        mask = create_future_mask(
            o       = o, 
            r       = r, 
            sync    = mask_sync
        )[0,:,:].int() # As there is only 1 variable/variables are flattened, an extra dim is created by the masking function
    else:
        mask = create_subsequence_mask(
            o       = o,
            r       = r,
            stateful= mask_stateful,
            sync    = mask_sync
        )[0,:,:].int() # As there is only 1 variable/variables are flattened, an extra dim is created by the masking function
    mssg.print(f"Before shape adjustment | batch ~ {batch.shape} | batch_masks ~ {bms.shape} | mask ~ {mask.shape}")
    mssg.final()
    # Restore mssg
    mssg.function = func 
    mssg.level -= 1
    return mask

In [ ]:
#| export
def check_batch_masks(
    batch,
    batch_masks,
    mssg
):
    # Setup mssg 
    func = mssg.function
    mssg.level += 1
    mssg.initial_(ut.funcname())
    # Ensure same number of masks than batches
    batch_masks = batch_masks[:batch.shape[0]]
    # Ensure same size for mask and batch
    if batch_masks.shape[1] < batch.shape[2]:
        # Should be bigger
        mssg.print_error(f"Invalid batch masks shape {batch_masks.shape}.")
        batch_masks = torch.nn.functional.pad(
            batch_masks, 
            (0, batch.shape[2]-batch_masks.shape[1])
        )
        mssg.print_error(f"Modified to {batch_masks.shape} adding 0's.")
    elif batch_masks.shape[1] > batch.shape[2]:
        # Should be lower
        mssg.print_error(f"Invalid batch masks shape {batch_masks.shape}.")
        batch_masks = batch_masks[:,:batch.shape[2]]
        mssg.print_error(f"Cropped to {batch_masks.shape} adding 0's.")
    # All right!
    mssg.print(f"batch~{batch.shape} | batch_masks~{batch_masks.shape}.")
    mssg.final()
    # Restore mssg 
    mssg.function = func 
    mssg.level -= 1
    return batch_masks

def check_mask(
    batch, batch_masks,mask, mssg
):
    # Setup mssg 
    func = mssg.function
    mssg.level += 1
    mssg.initial_(ut.funcname())
    
    # Should have as masks as batch_masks
    if mask.shape[1] < batch_masks.shape[1]:
        # Should be bigger
        mssg.print_error(f"Invalid mask shape {mask.shape}")
        mask = torch.nn.functional.pad(
            mask, 
            (0, batch_masks.shape[1]-mask.shape[1])
        )
        mssg.print_error(f"Modified to {mask.shape} adding 0's.")
    elif mask.shape[1] > batch_masks.shape[1]:
        # Should be lower
        mssg.print_error(f"Invalid mask shape {mask.shape}")
        mask = mask[:,:batch_masks.shape[1]]
        mssg.print_error(f"Cropped to {mask.shape} by the end")
    # All right!
    # Restore mssg 
    mssg.function = func 
    mssg.level -= 1
    return mask

In [ ]:
#| export
def moment_set_masks(
    batch, 
    batch_masks,
    window_mask_percent     : float     = 0.3,
    mssg                    : ut.Mssg   = ut.Mssg(),
    use_moment_masks        : bool      = False,
    mask_stateful           : bool      = False,
    mask_future             : bool      = False,
    mask_sync               : bool      = False
):
    # Setup mssg
    func = mssg.function
    mssg.level +=1
    mssg.initial_(ut.funcname())
    # Ensure batch sizes
    bms = check_batch_masks(batch, batch_masks, mssg)
    mssg.level += 1
    mssg.print(f"batch ~ {batch.shape} | batch_masks ~ {bms.shape}")
    if bms.shape[0] > batch.shape[0]: bms = bms[:batch.shape[0]]
    mssg.level -= 1
    mssg.print(f"window_mask_percent {window_mask_percent} | batch ~ {batch.shape}")
    # Move tensors to the same computation device
    device = batch.device if batch.device != "cpu" else bms.device
    batch = batch.to(device)
    bms = bms.to(device)
    # Get masks
    window_mask_percent = float(window_mask_percent)
    if use_moment_masks: # Use moment generator
        mask = get_mask_moment(
            batch       = batch,
            batch_masks = bms, 
            r           = window_mask_percent,
            mssg        = mssg
        )
    else: # Use tsai mask generators
        mask = get_mask_tsai(
            batch           = batch,
            batch_masks     = bms,
            r               = window_mask_percent,
            mask_stateful   = mask_stateful,
            mask_future     = mask_future,
            mask_sync       = mask_sync,
            mssg            = mssg
        )
    # Check sizes
    mask    = check_mask(batch, bms, mask, mssg)
    mssg.print(f"mask~{mask.shape}")
    mssg.print(f"batch_masks~{bms.shape}")
    #mssg.print_error(f"mask: {mask}")
    #mssg.print_error(f"batch_masks: {batch_masks}")
    mssg.final()
    # Restore mssg
    mssg.level -= 1
    mssg.function = func
    return mask, bms

In [ ]:
#| export
def fine_tune_moment_eval_step_(
    self : Encoder, 
    batch,
    mse_metric, 
    rmse_metric,
    mae_metric,
    smape_metric
):
    # Setup mssg
    func = self.mssg.function 
    self.mssg.level += 1
    self.mssg.initial(ut.funcname())
    # Setup masks
    window_size = batch.shape[2]
    device = "cpu" if self.cpu else torch.cuda.current_device()
    batch_masks = torch.ones((self.input.batch_size, window_size), device = device).long()
    mask, bms = moment_set_masks(
        batch               = batch,
        batch_masks         = batch_masks,
        window_mask_percent = self.input.window_mask_percent,
        mssg                = self.mssg,
        use_moment_masks    = self.use_moment_masks,
        mask_stateful       = self.mask_stateful,
        mask_future         = self.mask_future,
        mask_sync           = self.mask_sync
    )
    # -- Model evaluation
    self.model.eval()
    self._set_seed_(27)
    device = batch.device
    if self.cpu != "cpu": torch.cuda.synchronize()
    with torch.no_grad(), torch.cuda.device(device):
        self.mssg.print(f"Executing in device {device}")
        # Exec forward pass
        output, success = self.moment_safe_forward_pass(batch = batch, input_mask = bms, mask = mask)
        # If the execution of the forward pass was ok, compute the evaluation metrics
        if output is not None and success:
            self.mssg.print("Output is not None")
            # Compute loss
            loss = self.moment_compute_loss(batch, output, bms, mask)
            loss = loss if not hasattr(loss, 'item') else loss.item()
            # Get predictions/output and references / original
            predictions = output.reconstruction
            references  = batch
            # Ensure to avoid nan errors & ensure valid shape for metrics
            predictions, references = self.fine_tune_moment_eval_preprocess(predictions = predictions, references = references)
            # Move tensors to the computation device (cpu/gpu)
            predictions = predictions.to(device)
            references  = references.to(device)
            # Compute the different metrics
            mse_metric.add_batch(predictions=predictions, references = references)
            rmse_metric.add_batch(predictions=predictions, references = references)
            mae_metric.add_batch(predictions=predictions, references = references)
            smape_metric.add_batch(predictions=predictions, references = references)
        else:
            self.mssg.print("Output None or not successfull forward pass computation.")
    
    # Restore cuda
    self._restore_cuda_()
    self.mssg.final()
    # Restore mssg
    self.mssg.level -= 1
    self.mssg.function = func
    return mse_metric, rmse_metric, mae_metric, smape_metric, loss
Encoder.fine_tune_moment_eval_step_ = fine_tune_moment_eval_step_

In [ ]:
#| export
def fine_tune_moment_eval_(
    self      : Encoder,
    dl_eval   : DataLoader
):
    self.mssg.print_error("COMPUTING FINE TUNE MOMENT EVAL") #quitar al acabar depuración
    # Setup mssg
    func = self.mssg.function
    self.mssg.level += 1
    self.mssg.initial()
    # Evaluation metrics dict
    eval_results = {
        "mse": np.nan,
        "rmse": np.nan,
        "mae": np.nan,
        "smape": np.nan
    }
    # Select device
    device = "cpu" if self.cpu else torch.cuda.current_device()
    # Load metric
    mse_metric  = EvalMSE
    rmse_metric = EvalRMSE
    mae_metric  = EvalMAE
    smape_metric= EvalSMAPE
    losss       = []
    total_loss  = 0.0
    num_evaluation_steps = len(dl_eval)
    self.model = self.model.to(device)
    
    # Evaluation loop
    self.model.eval()
    progress_bar = tqdm(range(num_evaluation_steps))
    for batch in dl_eval:
        batch = batch.to(device)
        (
            mse_metric, 
            rmse_metric, 
            mae_metric, 
            smape_metric,
            loss
        ) = self.fine_tune_moment_eval_step_(
            batch       = batch, 
            mse_metric  = mse_metric, 
            rmse_metric = rmse_metric,
            mae_metric  = mae_metric,
            smape_metric= smape_metric
        )
        losss.append(loss)
        progress_bar.update(1)
    progress_bar.close()
    
    # Update evaluation metrics
    eval_results ["loss"]   = np.nanmean(np.array(losss))
    try:
        mse   = mse_metric.compute(squared = False)
        rmse  = rmse_metric.compute(squared = True)
        mae   = mae_metric.compute()
        smape = smape_metric.compute()
        eval_results ["mse"]    = mse['mse']
        eval_results ["rmse"]   = rmse['mse']
        eval_results ["mae"]    = mae['mae']
        eval_results ["smape"]  = smape['smape']
    except:
        #raise ValueError("Could not compute metrics. Already used add?")
        warnings.warn("Could not compute metrics. Already used add?")

    self.mssg.print_error(f"Eval results: {eval_results}.")
    self.model.train()
    self.mssg.final()
    # Restore mssg
    self.mssg.function = func
    self.mssg.level -= 1
    return eval_results
Encoder.fine_tune_moment_eval_ = fine_tune_moment_eval_

In [ ]:
#| export
def fine_tune_moment_eval_mix_windows_(
    self      : Encoder
):
    self.mssg.print_error("COMPUTING FINE TUNE MOMENT EVAL") #quitar al acabar depuración
    # Setup mssg
    func = self.mssg.function
    self.mssg.level += 1
    self.mssg.initial()
    # Evaluation metrics dict
    eval_results = {
        "loss"  : np.nan,
        "mse"   : np.nan,
        "rmse"  : np.nan,
        "mae"   : np.nan,
        "smape" : np.nan
    }
    # Select device
    device = "cpu" if self.cpu else torch.cuda.current_device()
    # Load metric
    mse_metric  = EvalMSE
    rmse_metric = EvalRMSE
    mae_metric  = EvalMAE
    smape_metric= EvalSMAPE
    losss       = []
    total_loss  = 0.0
    self.model = self.model.to(device)
    
    # Evaluation loop
    self.model.eval()
    num_batches = self.input.data.num_batches('valid')
    progress_bar = tqdm(range(num_batches))
    self.mssg.print(f"Num validation steps: {num_batches}")
    for batch in self.input.data.valid_batches():
        batch = batch.to(device)
        (
            mse_metric, 
            rmse_metric, 
            mae_metric, 
            smape_metric,
            loss
        ) = self.fine_tune_moment_eval_step_(
            batch       = batch, 
            mse_metric  = mse_metric, 
            rmse_metric = rmse_metric,
            mae_metric  = mae_metric,
            smape_metric= smape_metric
        )
        losss.append(loss)
        progress_bar.update(1)
    progress_bar.close()
    
    # Update evaluation metrics
    eval_results ["loss"]   = np.nanmean(np.array(losss))
    try:
        mse   = mse_metric.compute(squared = False)
        rmse  = rmse_metric.compute(squared = True)
        mae   = mae_metric.compute()
        smape = smape_metric.compute()
        eval_results ["mse"]    = mse['mse']
        eval_results ["rmse"]   = rmse['mse']
        eval_results ["mae"]    = mae['mae']
        eval_results ["smape"]  = smape['smape']
    except:
        #raise ValueError("Could not compute metrics. Already used add?")
        warnings.warn("Could not compute metrics. Already used add?")

    self.mssg.print_error(f"Eval results: {eval_results}.")
    self.model.train()
    self.mssg.final()
    # Restore mssg
    self.mssg.function = func
    self.mssg.level -= 1
    return eval_results
Encoder.fine_tune_moment_eval_mix_windows_ = fine_tune_moment_eval_mix_windows_

In [ ]:
#| export
def fine_tune_moment_train_loop_step_(
    self : Encoder,
    batch, 
    batch_masks,
    criterion                   = torch.nn.MSELoss,
    use_moment_masks    : bool  = False,
): 
    # Configure mssg
    func = self.mssg.function
    self.mssg.level += 1
    self.mssg.initial_(ut.funcname())
    # Get masks
    self.mssg.print("Get the masks")    
    mask, bms = moment_set_masks(batch, batch_masks, self.input.window_mask_percent, self.mssg, use_moment_masks, self.mask_stateful, self.mask_future, self.mask_sync)
    # Move tensors to the device
    device = torch.cuda.current_device() if not self.cpu else "cpu"
    batch   = batch.to(device)
    bms     = bms.to(device) 
    mask    = mask.to(device)
    self.model = self.model.to(device)
    self.mssg.level += 1
    self.mssg.print(f"batch ~ {batch.shape} | batch_masks ~ {bms.shape} | mask ~ {mask.shape}")
    for param in self.model.parameters(): param = param.to(device)
    self.mssg.print(f"Batch device: {batch.device} | mask device: {mask.device} | batch_masks device: {bms.device}")
    self.mssg.level -= 1
    # Compute a forward pass
    device = batch.device
    with torch.cuda.device(device):
        self.mssg.print(f"Executing in device {device}")
        output, success  = self.moment_safe_forward_pass(
            batch               = batch, 
            input_mask          = bms,  
            mask                = mask, 
            continue_if_fail    = True
        )
    # Compute output loss
    if output is None or not success:
        # If fail, put to nan & raise error | TODO: Check
        mssg = f"Execution failed: Output is None? {output} | Success {success}"
        loss = np.nan
        self.mssg.print(mssg)
        raise ValueError(f"Execution failed: Output is None.")
    else: 
        # Compute the loss
        self.mssg.print("Output correctly obtained, compute loss")
        loss = self.moment_compute_loss(
            batch       = batch,
            output      = output,
            input_mask  = bms,
            mask        = mask
        )
    # Restore mssg
    self.mssg.final()
    self.mssg.level -= 1
    self.mssg.function = func
    return loss
Encoder.fine_tune_moment_train_loop_step_ = fine_tune_moment_train_loop_step_

In [ ]:
#| export
def config_optim(
    self : Encoder,
    dl_train,
    num_training_steps
):
    if self.optim.optimizer is None: 
        self.optim.optimizer    = torch.optim.AdamW(self.model.parameters(), self.optim.lr.lr)
    if self.optim.lr.flag:
        self.optim.lr.scheduler = self.setup_scheduler(
            dl_train            = dl_train, 
            optimizer           = self.optim.optimizer, 
            num_training_steps  = num_training_steps 
        )
    else:
        self.optim.lr.scheduler = None
Encoder.config_optim = config_optim

In [ ]:
#| export
def fine_tune_moment_train_(
    self                            : Encoder,
    dl_train                        : DataLoader,
    ds_train                        : pd.DataFrame,
    use_moment_masks                : bool              = True,
    save_best_or_last               : bool              = True, #If true, save best else save last
    loss_pre                        : Optional[float]   = None
):
    # Setup mssg
    self.mssg.level += 1
    func = self.mssg.function
    self.mssg.initial_(ut.funcname())
    # Output param
    losses = []
    epoch_losses = []
    # Select device
    device = "cpu" if self.cpu else torch.cuda.current_device()
    # Optimizer and learning rate scheduler
    num_training_steps = self.num_epochs* len(dl_train)
    self.config_optim( dl_train = dl_train, num_training_steps=num_training_steps)
    # Masks
    window_size = ds_train.shape[2]
    batch_masks = torch.ones((self.input.batch_size, window_size), device = device).long()
    # Training loop
    self.mssg.print("Training loop")
    self.mssg.level += 1
    self.mssg.print(f"Fine tune loop | batch_masks~{batch_masks.shape}")
    progress_bar = tqdm(range(num_training_steps))
    self.mssg.level -= 1
    self.mssg.print(f"num_epochs {self.num_epochs} | n_batches {len(dl_train)}")
    if save_best_or_last:
        epoch_loss_mean         = np.nan
        self.best_epoch         = -1
        self.best_model_state   = {k: v.clone().detach().cpu() for k, v in self.model.state_dict().items()}
        if loss_pre is not None:
            self.best_loss  = loss_pre
            losses.append(self.best_loss)
        else:
            self.best_loss = np.inf

    for epoch in range(self.num_epochs):
        epoch_losses = []
        for i, batch in enumerate(dl_train):
            # Loop step
            self.mssg.print(f"batch {i} ~ {batch.shape} | epoch {epoch} | train {i+epoch} of {num_training_steps} | Before loop step")
            loss  = self.fine_tune_moment_train_loop_step_(
                    batch            = batch,
                    batch_masks      = batch_masks, 
                    use_moment_masks = use_moment_masks,
                )
            # Execute optimizer and get loss
            try: 
                self.mssg.print(f"fine_tune_moment_train | batch {i} ~ {batch.shape} | epoch {epoch} | train {i+epoch} of {num_training_steps} | Loss backward | After loop step ")
                self.optim.optimizer.zero_grad()
                if hasattr(loss, 'item'):
                    epoch_losses.append(loss.item())
                    loss.backward()
                else:
                    epoch_losses.append(loss)
                self.optim.optimizer.step()
            except Exception as e: 
                self.mssg.print(f"fine_tune_moment_train | batch {i} ~ {batch.shape} | epoch {epoch} | train {i+epoch} of {num_training_steps} | Loss backward failed: {e}")
                self.optim.optimizer.zero_grad()
                self.optim.optimizer.step()
            if self.optim.lr.flag: self.optim.lr.scheduler.step()
            progress_bar.update(1)
        # Attatch the mean in all batches
        epoch_losses    = np.array(epoch_losses)
        epoch_loss_mean = np.nanmean(epoch_losses)
        losses.append(epoch_loss_mean)
        if save_best_or_last and epoch_loss_mean < self.best_loss:
            self.mssg.print_error(f"Best Loss {self.best_loss} -> {epoch_loss_mean}")
            self.mssg.print_error(f"Best epoch {epoch}")
            self.best_epoch         = epoch
            self.best_loss          = epoch_loss_mean 
            self.best_model_state   = {k: v.clone().detach().cpu() for k, v in self.model.state_dict().items()}
    progress_bar.close()
    # Get the best version of the model
    if save_best_or_last:
        self.model.load_state_dict(self.best_model_state)
        self.mssg.print_error(f"Loaded status in the best epoch ({self.best_epoch})")
    self.mssg.final()
    # Restore mssg
    self.mssg.level -= 1
    self.mssg.function = func 
    return losses, self.model
Encoder.fine_tune_moment_train_ = fine_tune_moment_train_

In [ ]:
#| export
#-- moving to set_train_and_eval_dataloaders
def prepare_train_and_eval_dataloaders(
    X                   : Union [ List [ List [ List [ float ]]], List [ float ], pd.DataFrame ],
    batch_size          : int,
    n_windows           : int       = None,
    n_windows_percent   : int       = None,
    training_percent    : int       = 0.4,
    validation_percent  : int       = 0.3,
    shot                : bool      = False,
    eval_pre            : bool      = False,
    eval_post           : bool      = False,
    mssg                : ut.Mssg   = ut.Mssg()
):
    dl_eval  = None
    ds_train = None,
    dl_train = None
    func = mssg.function
    mssg.level += 1
    mssg.function = f"{mssg.function} | prepare_train_and_eval_dataloaders"
    if n_windows is None and n_windows_percent is None:
        train_split_index = min(X.shape[0], np.ceil(training_percent * X.shape[0]))
        eval_split_index = min(X.shape[0], np.ceil(validation_percent * X.shape[0]))
    else:
        train_split_index = min(X.shape[0], np.ceil(training_percent * n_windows)) if n_windows is not None else np.ceil(training_percent * n_windows_percent * X.shape[0])
        eval_split_index = min(X.shape[0], np.ceil(validation_percent * n_windows)) if n_windows is not None else np.ceil(validation_percent * n_windows_percent * X.shape[0])
    
    train_split_index = int(train_split_index)
    eval_split_index = int(eval_split_index)
    if shot: 
        mssg.print(f"Selecting ds train | {train_split_index} windows")
        ds_train = X[:train_split_index]
    if eval_pre or eval_post: 
        mssg.print(f"Selecting validation train | {eval_split_index} windows")
        ds_test  = torch.from_numpy(X[:eval_split_index]).float()
    # -- Select only the small percentage for few-shot
    if shot:
        mssg.print(f"Train DataLoader | Random windows")
        mssg.verbose -= 1
        ds_train = random_windows(ds_train, n_windows, n_windows_percent, mssg = mssg)
        mssg.verbose += 1
        ds_train = ds_train.float()
        # Create the dataloader
        mssg.print(f"Train DataLoader | DataLoader")
        dl_train = DataLoader(ds_train, batch_size = batch_size, shuffle = True)
    if eval_pre or eval_post: 
        mssg.print(f"Validation DataLoader")
        dl_eval  = DataLoader(ds_test, batch_size = batch_size, shuffle = False)
    mssg.level -= 1
    mssg.function = func
    return dl_eval, dl_train, ds_test, ds_train

In [ ]:
#| export
def fine_tune_moment_single_(
    self                : Encoder,
    eval_pre            : bool = False,
    eval_post           : bool = False,
    shot                : bool = True,
    sample_id           : int  = 0,
    use_moment_masks    : bool = False,
    register_errors     : bool = True,
    save_best_or_last   : bool = True #If true, save best else save last
):
    # Setup mssg
    self.mssg.level += 1
    func = self.mssg.function
    self.mssg.initial_(ut.funcname())
    # Output & aux parameters
    t_shot              = 0
    t_eval_1            = 0
    t_eval_2            = 0
    losses              = []
    eval_results_pre    = {}
    eval_results_post   = {}
    error_val           = 0
    if self.time_flag: timer = ut.Time(mssg = self.mssg)
    self.mssg.print(f"fine_tune_moment_single | Prepare the dataset | X ~ {self.input.data[sample_id].shape}")
    # Prepare the dataset
    (
        dl_eval, dl_train, 
        ds_eval, ds_train 
    ) = self.set_train_and_eval_dataloaders(sample_id)
    # Evaluation % training
    try:
        self.mssg.print(f"Eval wlen {self.input.data[sample_id].shape[2]} | Lengths list: {self.window_sizes}")
        # Previous evaluation
        loss_pre = None
        if eval_pre:
            torch.cuda.synchronize()
            self.mssg.print_error(f"Eval Pre | wlen {self.input.data[sample_id].shape[2]}")
            if self.time_flag: timer.start()
            eval_results_pre    = self.fine_tune_moment_eval_(dl_eval = dl_eval)
            if self.time_flag: 
                timer.end()
                t_eval_1 = timer.duration()
                timer.show(verbose = self.mssg.verbose)
            torch.cuda.synchronize()
            #self.mssg.print_error(f"Eval pre after computation: {eval_results_pre}")
        # Training (fine-tuning)
        if shot:
            if eval_pre: loss_pre = eval_results_pre["loss"]
            torch.cuda.synchronize()
            if self.time_flag: timer.start()
            self.mssg.print(f"Train | wlen {self.input.data[sample_id].shape[2]}")
            try:
                losses, self.model                  = self.fine_tune_moment_train_(
                    dl_train                        = dl_train,
                    ds_train                        = ds_train,
                    use_moment_masks                = use_moment_masks,
                    save_best_or_last               = save_best_or_last,
                    loss_pre                        = loss_pre
                )
                torch.cuda.synchronize()

                if self.time_flag:
                    timer.end()
                    t_shot = timer.duration()
                    timer.show()
            except Exception as e:
                self.mssg.print(f"fine_tune_moment_single | Train | Window {self.input.shape[2]} not valid | {e}")
                traceback.print_exc()
                raise(e)
        # After evaluation
        if eval_post:    
            torch.cuda.synchronize()

            self.mssg.print(f"fine_tune_moment_single | Eval Post | wlen {self.input.shape[2]}")
            if self.time_flag: timer.start()
            eval_results_post = self.fine_tune_moment_eval_(dl_eval=dl_eval)
            #self.mssg.print_error(f"Eval_results_post = {eval_results_post}")
            if self.time_flag:
                timer.end()
                t_eval_2 = timer.duration()
                if self.mssg.verbose > 0: 
                    timer.show()
            torch.cuda.synchronize()
        
        if self.mssg.verbose > 0 and (eval_pre or eval_post): 
            self.show_eval_stats(
                # Wether computed or not pre & post errors
                eval_pre        = eval_pre, 
                eval_post       = eval_post, 
                # Results
                eval_stats_pre  = eval_results_pre,
                eval_stats_post = eval_results_post,
                # Function name
                func_name       = ut.funcname()
            )

    except Exception as e:
        # Save error if failed
        if register_errors:
            error_val   = 1
            window      = self.input.data[sample_id].shape[2]
            self.mssg.print_error(f"Registering error in DataFrame | window: {window} | error: {e}")
            error       = {"window": window, "error": e}
            self.errors = pd.concat([self.errors, pd.DataFrame([error])])
            display(self.errors)
        else: 
            raise(e)
    #self.mssg.print_error(f"Final Eval pre: {eval_results_pre}")
    self.mssg.final()
    # Restore mssg
    self.mssg.level -= 1
    self.mssg.function = func
    return losses, eval_results_pre, eval_results_post, t_shot, t_eval_1, t_eval_2, self.model, error_val

Encoder.fine_tune_moment_single_ = fine_tune_moment_single_

In [ ]:
#| export
def fine_tune_moment_train_mix_windows_(
    self                : Encoder,
    use_moment_masks    : Optional[bool]    = True,
    save_best_or_last   : Optional[bool]    = True,
    loss_pre            : Optional[float]   = None
):
    # Setup mssg
    self.mssg.level += 1
    func = self.mssg.function
    self.mssg.initial_(ut.funcname())
    # Output param
    losses = []
    epoch_losses = []
    # Select device
    device = "cpu" if self.cpu else torch.cuda.current_device()
    # Optimizer and learning rate scheduler
    num_batches = sum(1 for _ in self.input.data.train_batches())
    num_training_steps = self.num_epochs * num_batches
    self.mssg.print_error(f"Num training steps: {self.num_epochs}*{num_batches} = {num_training_steps}")
    if (num_training_steps == 0): 
        raise ValueError("Please check the configuration, the train dataset results in 0 batches.")
    if self.optim.optimizer is None:
        self.optim.optimizer = torch.optim.AdamW(self.model.parameters(), self.optim.lr.lr)
    
    steps_per_epoch = self.input.data.num_batches('train')
    if self.optim.lr.flag:
        if not self.optim.lr.num_warmup_steps:
            lr_scheduler_num_warmup_steps = 0.02*num_training_steps 
        lr_scheduler_max_lr = 5-10*self.optim.lr.lr if lr_scheduler_max_lr is None else lr_scheduler_max_lr
        match self.optim.lr.name:
            case "OneCycleLR":
                self.optim.lr.scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer           = self.optim.optimizer,
                    max_lr              = lr_scheduler_max_lr,
                    epochs              = self.num_epochs,
                    steps_per_epoch     = steps_per_epoch
                )
            case _:
                self.optim.lr.scheduler = get_scheduler(
                    name                = self.optim.lr.name,
                    optimizer           = self.optim.optimizer,
                    num_warmup_steps    = lr_scheduler_num_warmup_steps,
                    num_training_steps  = num_training_steps
                )
    else:
        self.optim.lr.scheduler = None
    progress_bar = tqdm(range(num_training_steps))
    
    if save_best_or_last:
        epoch_loss_mean         = np.nan
        self.best_epoch         = -1
        self.best_model_state   = {k: v.clone().detach().cpu() for k, v in self.model.state_dict().items()}
        if loss_pre is not None:
            self.best_loss      = loss_pre
            losses.append(self.best_loss)
    
    i = 0
    for epoch in range (self.num_epochs):
        epoch_losses = []
        for batch in self.input.data.train_batches():
            i+=1
            window_size = batch.shape[2]
            self.window_sizes.append(window_size)
            batch_masks = torch.ones((self.input.batch_size, window_size), device = device).long()
            loss  = self.fine_tune_moment_train_loop_step_(
                batch            = batch,
                batch_masks      = batch_masks, 
                use_moment_masks = use_moment_masks,
            )
            # Execute optimizer and get loss
            try: 
                self.mssg.print_error(f"fine_tune_moment_train (1) | batch ~ {batch.shape} | epoch {epoch} | train {i+epoch} of {num_training_steps} | Loss backward | After loop step ")
                self.optim.optimizer.zero_grad()
                if hasattr(loss, 'item'):
                    epoch_losses.append(loss.item())
                    loss.backward()
                else:
                    epoch_losses.append(loss)
                self.optim.optimizer.step()
            except Exception as e: 
                self.mssg.print_error(f"fine_tune_moment_train (2) | batch ~ {batch.shape} | epoch {epoch} | train {i+epoch} of {num_training_steps} | Loss backward failed: {e}")
                self.optim.optimizer.zero_grad()
                self.optim.optimizer.step()
            if self.optim.lr.flag: self.optim.lr.scheduler.step()
            progress_bar.update(1)
            if np.isnan(epoch_losses[-1]): 
                raise ValueError("Loss is NaN")
        self.mssg.print_error(f"Epoch losses: {epoch_losses}")
        epoch_losses = np.array(epoch_losses)
        epoch_loss_mean = np.nanmean(epoch_losses)
        losses.append(epoch_loss_mean)
        if save_best_or_last and epoch_loss_mean < self.best_loss:
            self.mssg.print_error(f"Best Loss {self.best_loss} -> {epoch_loss_mean}")
            self.mssg.print_error(f"Best epoch {epoch}")
            self.best_epoch = epoch
            self.best_loss  = epoch_loss_mean 
            self.best_model_state = {k: v.clone().detach().cpu() for k, v in self.model.state_dict().items()}
    progress_bar.close()
    # Get the best version of the model 
    if save_best_or_last and self.best_model_state:
        self.model.load_state_dict(self.best_model_state)
        self.mssg.print_error(f"Best epoch: {self.best_epoch}")
    self.mssg.final()
    # Restore mssg
    self.mssg.level -= 1
    self.mssg.function = func
    return losses, self.model 

Encoder.fine_tune_moment_train_mix_windows_ = fine_tune_moment_train_mix_windows_

In [ ]:
#| export 
def fine_tune_moment_mix_windows(
    self                : Encoder,
    eval_pre            : bool = False,
    eval_post           : bool = False,
    shot                : bool = True,
    sample_id           : int  = 0,
    use_moment_masks    : bool = False,
    register_errors     : bool = True,
    save_best_or_last   : bool = True #If true, save best else save last
):
    # Setup mssg 
    self.mssg.level += 1
    func = self.mssg.function
    self.mssg.initial_(ut.funcname())
    # Return parameters
    losses              = []
    eval_results_pre    = {}
    eval_results_post   = {} 
    t_shot              = 0.0
    t_eval_1            = 0.0
    t_eval_2            = 0.0
    error_val           = 0
    
    if self.time_flag: timer = ut.Time(mssg = deepcopy(self.mssg))
    
    # Go!
    try: 
        #self.window_sizes = set()
        # Pre-validation
        if eval_pre: 
            if self.cpu != "cpu" and self.time_flag:
                torch.cuda.synchronize()
            if self.time_flag: timer.start()
            eval_results_pre = self.fine_tune_moment_eval_mix_windows_()
            if self.cpu != "cpu" and self.time_flag:
                torch.cuda.synchronize()
            if self.time_flag: 
                timer.end()
                t_eval_1 = timer.duration()
        # Training 
        if shot: 
            if self.cpu != "cpu" and self.time_flag:
                torch.cuda.synchronize()
            if self.time_flag: timer.start()
            loss_pre = None if not eval_pre else eval_results_pre["loss"]
            losses, self.model = self.fine_tune_moment_train_mix_windows_(use_moment_masks, save_best_or_last, loss_pre = loss_pre)
            if self.cpu != "cpu" and self.time_flag:
                torch.cuda.synchronize()
            if self.time_flag: 
                timer.end()
                t_shot = timer.duration()
        
        
        
        # Post-validation
        if eval_post: 
            if self.cpu != "cpu" and self.time_flag:
                torch.cuda.synchronize()
            if self.time_flag: timer.start()
            eval_results_post = self.fine_tune_moment_eval_mix_windows_()
            if self.cpu != "cpu" and self.time_flag:
                torch.cuda.synchronize()
            if self.time_flag: 
                timer.end()
                t_eval_2 = timer.duration()
    except Exception as e:
        if register_errors:
            error_val = 1
            self.mssg.print_error(f"Registering error | {e}")
            traceback.print_exc()
            error = {"window": self.input.data.window_sizes, "error": e}
            self.errors = pd.concat([self.errors, ])
            display(self.errors)
        else: 
            raise(e)
    self.mssg.final()
    # Restore mssg 
    self.mssg.function = func 
    self.mssg.level -= 1
    return losses, eval_results_pre, eval_results_post, t_shot, t_eval_1, t_eval_2, self.model, error_val

Encoder.fine_tune_moment_mix_windows = fine_tune_moment_mix_windows

In [ ]:
#| export
def fine_tune_moment_(
    self                : Encoder, 
    eval_pre            : bool = False, 
    eval_post           : bool = False, 
    shot                : bool = False,
    time_flag           : bool = None,
    use_moment_masks    : bool = None,
    register_errors     : bool = True,
    save_best_or_last   : bool = True, #If true, save best else save last
    mix_windows         : bool = True
):   
    # Setup mmsg & use_moment_masks
    func = self.mssg.function 
    self.mssg.level += 1
    self.mssg.initial(ut.funcname())
    self.time_flag = self.time_flag if time_flag is None else time_flag
    self.use_moment_masks = self.use_moment_masks if use_moment_masks is None else use_moment_masks
    # Return & aux values
    lossess             = []
    best_epochs         = []
    eval_results_pre    = {}
    eval_results_post   = {}
    t_shots             = []
    t_shot              = 0
    t_evals             = []
    t_eval              = 0
    # Check
    if self.input._data.__class__ != ut.WindowedDataset and self.input.size is None:
        self.mssg.print(f"Windows: {len(self.input._data)}")
        raise ValueError(f"Invalid number of windows: {self.input.size}")
    self.mssg.print(f"Processing {self.input.size} datasets : {self.input.shape}")
    # Build optimizer
    if self.optim.optimizer is None: 
        self.mssg.print(f"Setting up optimizer as AdamW")
        self.optim.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.optim.lr.lr)
    
    if mix_windows:
        ( 
            losses, 
            eval_results_pre_,
            eval_results_post_,
            t_shot_,
            t_eval_1,
            t_eval_2, 
            self.model, 
            error_val 
        ) = self.fine_tune_moment_mix_windows(
            eval_pre            = eval_pre, 
            eval_post           = eval_post, 
            shot                = shot, 
            use_moment_masks    = use_moment_masks, 
            register_errors     = register_errors, 
            save_best_or_last   = save_best_or_last
        )
    else:
        # Evaluate the model for each window in the windowed dataset
        for i in range(self.input.size):
            window_size = self.input._data[i].shape[2]
            self.mssg.print_error(f"Processing wlen {window_size} | wlens {self.window_sizes} | i {i+1}/{self.input.size}")
            self.window_sizes.add(window_size)
            self.ws = window_size
            ( 
                losses, 
                eval_results_pre_, 
                eval_results_post_, 
                t_shot_, 
                t_eval_1, 
                t_eval_2, 
                self.model,
                error_val
            ) =  self.fine_tune_moment_single_(
                eval_pre            = eval_pre, 
                eval_post           = eval_post, 
                shot                = shot, 
                sample_id           = i, 
                use_moment_masks    = use_moment_masks, 
                register_errors     = register_errors, 
                save_best_or_last   = save_best_or_last
            )
    if (error_val == 0): lossess.append(losses)
    if (eval_pre and error_val == 0): 
        if eval_results_pre == {}: 
                eval_results_pre = {
                key:[eval_results_pre_[key]] for key in eval_results_pre_.keys()
            }
        else:
            for key in eval_results_pre_.keys():
                self.mssg.print(eval_results_pre_[key])
                eval_results_pre[key] += [eval_results_pre_[key]]
        #self.mssg.print_error(f"About to concat {eval_results_post_} to {eval_results_post}")
    if (eval_post and error_val == 0):
        if eval_results_post == {}: 
            eval_results_post = {
                key:[eval_results_post_[key]] for key in eval_results_post_.keys()
            }
        else:
            for key in eval_results_post_.keys():
                self.mssg.print(eval_results_post_[key])
                eval_results_post[key] += [eval_results_post_[key]]
        #self.mssg.print_error(f"After concat {eval_results_post}")
    t_shots.append(t_shot_)
    if eval_pre: t_evals.append(t_eval_1)
    if eval_post: t_evals.append(t_eval_2)
    #eval_pre = False
    if save_best_or_last:
        self.mssg.print_error(f"best_epoch: {self.best_epoch}")
        best_epochs.append(self.best_epoch)
    t_shot = sum(t_shots)
    t_eval = sum(t_evals)
    self.eval_stats_pre = eval_results_pre
    self.eval_stats_post = eval_results_post
    self.mssg.final()
    self.window_sizes = list(set(self.window_sizes))
    # Restore mssg
    self.mssg.function = func
    self.mssg.level -= 1
    if register_errors: return lossess, eval_results_pre, eval_results_post, t_shots, t_shot, t_evals, t_eval, self.model, self.window_sizes, best_epochs, self.errors
    return lossess, eval_results_pre, eval_results_post, t_shots, t_shot, t_evals, t_eval, self.model, self.window_sizes, best_epochs

Encoder.fine_tune_moment_ = fine_tune_moment_

### MVP

In [ ]:
#| export
def fit_fastai(
    self     : Encoder,
    dl_train : DataLoader,
    lr_funcs : Tuple[Callable, ...] = (valley),  # steep
    show_plot: bool = False,
    n_cycles : Optional[int] = None,  # Allow dynamic calculation
    cycle_len : Optional[int] = None
) -> None:
    """
    Trains an `Encoder` model using different learning rate schedules, with support for
    automatic learning rate finding and early stopping.

    Args:
        self (Encoder): The `Encoder` instance to be trained.
        dl_train (DataLoader): The training DataLoader.
        lr_funcs (Tuple[Callable, ...], optional): Functions to suggest the learning rate 
            using `lr_find`. Defaults to `(valley,)`.
        show_plot (bool, optional): Whether to display the learning rate finder plot.
            Defaults to `False`.
        n_cycles (int, optional): Number of cycles for cyclical learning rate methods.
            If not provided, it is calculated as `n_epoch // cycle_len`.
        cycle_len (int, optional): Length of each cycle (in epochs) for cyclical 
            learning rate methods. If not provided, defaults to `1`.

    Raises:
        Exception: If `lr_find` fails to calculate the learning rate.

    Returns:
        None

    Behavior:
        - Uses the `lr_find` method to determine the optimal maximum learning rate (`lr_max`).
        - Calculates `pct_start` based on the number of warm-up steps relative to the total training steps.
        - Supports multiple learning rate schedules, including:
          - `OneCycleLR`: Trains with a one-cycle learning rate policy.
          - `FlatCosLR`: Trains with a flat cosine annealing schedule.
          - `SgdrLR`: Trains using stochastic gradient descent with restarts (SGDR).
          - Default `fit`: Trains using the basic fastai training loop if no scheduler matches.

    Notes:
        - Early stopping is enabled by default, monitoring `valid_loss` with a minimum delta of `1e-6` 
          and a patience of 10 epochs.
        - If `cycle_len` or `n_cycles` is not provided, default values are calculated to distribute 
          training evenly across cycles.

    Example:
        >>> encoder = Encoder(...)
        >>> dl_train = DataLoader(...)
        >>> encoder.fit_fastai(dl_train, lr_funcs=(valley,), show_plot=True, n_cycles=3, cycle_len=2)
    """
    # Setup mssg
    self.mssg.level += 1
    self.mssg.initial_("fit_fastai")
    # Callbacks
    cbs = [
        EarlyStoppingCallback(
            monitor="valid_loss", 
            min_delta=0.000001, 
            patience=10
        )
    ]
    
    # Learning rate finding
    lr = self.optim.lr if isinstance(self.optim.lr, float) else self.optim.lr.lr
    try:
        lr_max = self.model.lr_find(
            suggest_funcs = lr_funcs, 
            show_plot     = show_plot
        )
    except Exception as e:
        self.mssg.print_error(
            f"fit_fastai | error | {lr_funcs}"+
            f"lr | {lr} | {e}" 
        )
        raise 
    
    # Training steps and pct_start
    steps_per_epoch = len(dl_train)
    training_steps = self.num_epochs * steps_per_epoch
    warmup_steps = self.optim.lr.num_warmup_steps
    pct_start = warmup_steps / training_steps
    assert training_steps > 0, (
        f"Training steps must be greater than 0. Got: {training_steps}. "
        f"Check if num_epochs={self.num_epochs} or steps_per_epoch={steps_per_epoch} is invalid."
    )
    assert 0 <= warmup_steps <= training_steps, (
        f"Warmup steps must be between 0 and the total training steps. "
        f"Got: warmup_steps={warmup_steps}, training_steps={training_steps}."
    )
    assert 0 <= pct_start <= 1, (
        f"pct_start must be a valid percentage (0 <= pct_start <= 1). "
        f"Calculated: pct_start={pct_start} with warmup_steps={warmup_steps} and training_steps={training_steps}."
    )
    # Calculate n_cycles and cycle_len if not provided
    if cycle_len is None:
        cycle_len = 1  # Default to 1 epoch per cycle
    if n_cycles is None:
        n_cycles = self.num_epochs // cycle_len

    # Shared arguments
    args = {
        "n_epoch": self.num_epochs,
        "lr_max": lr_max,
        "pct_start": pct_start,
        "cbs": cbs
    }
    # change to print (error just print red, debugging this part)
    self.mssg.print_error(f"{args}")
    # Choose learning rate scheduler
    if (
        isinstance(self.optim.lr, float) or 
        not self.optim.lr.flag
    ):
        self.model.fit_one_cycle(**args)
    else:
        self.model.opt = self.optim.optimizer
        match self.optim.lr.name:
            case "OneCycleLr":
                self.model.fit_one_cycle(**args)
            case "FlatCosLR":
                self.model.fit_flat_cos(**args)
            case "SgdrLR":
                # Use calculated or provided n_cycles and cycle_len
                self.model.fit_sgdr(
                    n_cycles=n_cycles,
                    cycle_len=cycle_len,
                    lr_max=lr_max,
                    cbs=cbs
                )
            case _:
                self.model.fit(self.num_epochs)
    # Restore mssg
    self.mssg.final()
    self.mssg.level -= 1
Encoder.fit_fastai = fit_fastai    

In [ ]:
#| export
def fine_tune_mvp_single_(
    self            : Encoder,
    eval_pre        : bool  = False,
    eval_post       : bool  = False,
    shot            : bool  = False,
    show_plot       : bool  = False,
    sample_id       : int   = 0
):
    # Setup mssg
    func = self.mssg.func
    self.mssg.level+=1
    self.mssg.initial_(ut.funcname())
    #--- Setup object parameters ---
    self.show_plot = self.show_plot if show_plot is None else show_plot
    #--- Setup output values
    # Time
    t_shot = 0
    t_eval_1 = 0
    t_eval_2 = 0
    # Training losses
    losses = [],
    # Callbacks
    cbs = L()
    if self.use_wandb: cbs = L(CustomWandbCallback(log_preds=False))
    # Evaluation metrics & results
    #metrics_dict = get_metrics_dict(self.metrics, self.metrics_names)
    metrics_dict = self.metrics_dict #TODO: clean
    eval_results_pre = {key: 0.0 for key in metrics_dict.keys()}
    eval_results_post = {key: [] for key in metrics_dict.keys()}
    # Computation device
    device = "cpu" if self.cpu else torch.cuda.current_device()
    
    #### -------- Start -------####
    if self.time_flag : timer = ut.Time(mssg = self.mssg)
    # Split dataset and get the encoder input
    X = self.get_splits_(sample_id)
    self.mssg.print("About to set batch tfms")
    tfms = [ToFloat(), None]
    batch_tfms = [
        TSStandardize(
            by_sample       = self.norm_by_sample, 
            use_single_batch= self.norm_use_single_batch
        )
    ]
    # Get dataloaders
    dls = get_ts_dls(
        X           = X, 
        splits      = self.splits, 
        tfms        = tfms, 
        bs          = self.input.batch_size, 
        batch_tfms  = batch_tfms
    )
    # Configure callbacks
    if self.show_plot: 
        self.mssg.print("Show plot")
        display(dls.show_at(0))
        sgc = ShowGraphCallback2()
        self.model = ts_learner(
            dls, 
            InceptionTimePlus,
            cbs = cbs + sgc + MVP(
                r           = self.optim.lr if isinstance(self.optim.lr, float) else self.optim.lr.lr,
                window_size = X.shape[2]-1,
                future_mask = self.mask_future,
                target_dir  = './models',
                sync        = self.mask_sync,
                stateful    = self.mask_stateful,
                fname       = f'encoder_MVP',
            ),
            y_range = [X.min(), X.max()]#,
        )
    else:
        self.mssg.print("Don't show plot")
        self.model = ts_learner(
            dls, 
            InceptionTimePlus,
            cbs = cbs + mvp(
                r           = self.optim.lr if ( isinstance(self.optim.lr, float) or isinstance(self.optim.lr, int)) else self.optim.lr.lr,
                window_size = X.shape[2]-1,
                future_mask = self.mask_future,
                target_dir  = './models',
                sync        = self.mask_sync,
                stateful    = self.mask_stateful,
                fname       = f'encoder_MVP',
            ),
            y_range = [X.min(), X.max()]
        )
    
    self.model.to(device)
    self.mssg.print(f"Model Class {self.model.__class__} | Type: {type(self.model)}")
    
    # Eval - pre 
    if eval_pre:
        if self.time_flag: timer.start()
        self.mssg.print(f"Eval Pre | wlen {X.shape[2]} | Model: {self.model.__class__} | {type(self.model)} ")
        self.mssg.print(f"Model metrics: {self.model.metrics}")
        self.mssg.print(f"metrics: {self.metrics}")
        # Get metrics
        self.model.eval() # Evaluation mode
        results = self.validate_with_metrics_()
        # Format the results in json
        eval_results_pre = validate_with_metrics_format_results(
            results = results, 
            metrics = self.metrics, 
            metrics_names = self.metrics_names
        )
        self.mssg.print(f"Eval results pre with custom metrics: {eval_results_pre}")
        # Compute time
        if self.time_flag:
            timer.end()
            t_eval_1 = timer.duration()
            timer.show(verbose = self.mssg.verbose)
    # Train 
    if shot:
        if self.time_flag: timer.start()
        self.mssg.print(f"Training the model | window size {X.shape[2]} | X ~ {X.shape}")
        # Train the model 
        self.model.train() #Training mode
        # Train using the selected optimizer & scheduler
        self.fit_fastai(dls, show_plot = show_plot)
        losses = self.model.recorder.losses
        # Compute time
        if self.time_flag:
            timer.end()
            t_shot= timer.duration()
            timer.show(verbose = self.mssg.verbose)
    
    # Eval - post
    if eval_post:
        if self.time_flag: timer.start()
        self.mssg.print(f"Eval Pre | wlen {X.shape[2]}")
        # Get metrics
        self.model.eval() # Evaluation mode
        results = self.validate_with_metrics_()
        self.mssg.print(f"Format results | results~{len(results)}")
        eval_results_post = validate_with_metrics_format_results(
            results = results, metrics = self.metrics, metrics_names = self.metrics_names)
        #self.model.validate()
        self.mssg.print(f"Eval results post with custom metrics: {eval_results_post}")
        if self.time_flag:
            timer.end()
            t_eval_2 = timer.duration()
            timer.show(verbose = self.mssg.verbose)
    self.mssg.final()
    # Restore mssg
    self.mssg.level += 1
    self.mssg.function = func
    return losses, eval_results_pre, eval_results_post, t_shot, t_eval_1, t_eval_2, self.model
Encoder.fine_tune_mvp_single_ = fine_tune_mvp_single_

In [ ]:
#| export
def fine_tune_mvp_(
    self                    : Encoder,
    eval_pre                : bool  = True,
    eval_post               : bool  = True,
    shot                    : bool  = False,
    time_flag               : bool  = None,
    use_wandb               : bool  = None,
    analysis_mode           : str   = None,
    norm_by_sample          : bool  = None,
    norm_use_single_batch   : bool  = None,
    show_plot               : bool  = None
):
    # Setup mssg
    func = self.mssg.function 
    self.mssg.level += 1
    self.mssg.initial_("fine_tune_mvp_")
    # Setup encoder computation configuration
    self.time_flag      = self.time_flag if time_flag is None else time_flag
    self.use_wandb      = self.use_wandb if use_wandb is None else use_wandb
    self.analysis_mode  = self.analysis_mode if analysis_mode is None else analysis_mode
    self.norm_by_sample = self.norm_by_sample if norm_by_sample is None else norm_by_sample
    self.norm_use_single_batch = self.norm_use_single_batch if norm_use_single_batch is None else norm_use_single_batch
    # Return & aux values
    lossess             = []
    eval_results_pre    = {} # Only 1 (before fine-tune)
    eval_results_post   = {} # One per window size (after training)
    t_shots             = []
    t_shot              = 0
    t_evals             = []
    t_eval              = 0
    # Check
    if self.input._data.__class__ != ut.WindowedDataset and self.input.size is None:
        self.mssg.print(f"Windows: {len(self.input._data)}")
        raise ValueError(f"Invalid number of windows: {self.input.size}")
    self.mssg.print(f"Processing {self.input.size} datasets: {self.input.shape}")
    # Build optimizer
    if self.optim.optimizer is None: 
        self.mssg.print(f"Setting up optimizer as Adam")
        lr = self.optim.lr
        if (not isinstance(self.optim.lr, float)): 
            lr = self.optim.lr.lr,
       
        self.optim.optimizer = Adam(
            self.model.parameters(),
            lr = lr,
            mom = 0.0,
            sqr_mom = 0.99,
            eps = 1e-8,
            wd = 0.01,
            decouple_wd = True
        )
    self.mssg.print(f"Optimizer: {self.optim.optimizer}")
    # Compute model for each window in the windowed dataset
    for i in range(self.input.size):
        self.mssg.print(f"Processing wlen {self.input.shape[2]}")
        ( 
            losses, eval_results_pre_, eval_results_post_, t_shot_, t_eval_1, t_eval_2, self.model
        ) =  self.fine_tune_mvp_single_(eval_pre, eval_post, shot, sample_id = i, show_plot = show_plot)
        lossess.append(losses)
        if (eval_pre): eval_results_pre = eval_results_pre_
        
        if (eval_post): 
            if eval_results_post == {}: eval_results_post = {key:[] for key in eval_results_post_.keys()}
            #self.mssg.print_error(f"About to concat {eval_results_post_} to {eval_results_post}")
            for key in eval_results_post.keys():
                eval_results_post[key] += eval_results_post_[key]
            #self.mssg.print_error(f"After concat: {eval_results_post}")
        t_shots.append(t_shot_)
        if eval_pre: t_evals.append(t_eval_1)
        if eval_post: t_evals.append(t_eval_2)
        eval_pre = False
    t_shot = sum(t_shots)
    t_eval = sum(t_evals)
    self.mssg.final()
    # Restore mssg
    self.mssg.function = func
    self.mssg.level -= 1
    return lossess, eval_results_pre, eval_results_post, t_shots, t_shot, t_evals, t_eval, self.model

Encoder.fine_tune_mvp_ = fine_tune_mvp_ 

### Moirai

In [ ]:
#| export
def configure_optimizer_moirai(
    self                : Encoder, 
    dl_train            : DataLoader,
    weight_decay        : float = 1e-2
):
    """
    Configures the optimizer for fine-tuning the Moirai model.

    Args:
        enc (Encoder): The encoder containing the Moirai model to optimize.
        lr (float): Learning ratse.
        beta1 (float): Beta1 hyperparameter for AdamW optimizer.
        beta2 (float): Beta2 hyperparameter for AdamW optimizer.
        weight_decay (float): Weight decay for AdamW optimizer.

    Returns:
        dict: Dictionary with optimizer and learning rate scheduler configuration.
    """
    # Setup mssg
    self.mssg.level += 1
    func = self.mssg.function
    self.mssg.initial_(ut.funcname())
    # Configure optimizer
    if self.optim.optimizer is None:
        decay = set()
        no_decay = set()

        whitelist_params = (
            nn.Linear, 
            uni2ts.module.ts_embed.MultiInSizeLinear, 
            uni2ts.module.ts_embed.MultiOutSizeLinear, 
            uni2ts.module.position.attn_projection.LearnedProjection
        )
        
        blacklist_params = (
            nn.LayerNorm, 
            nn.Embedding, 
            uni2ts.module.norm.RMSNorm, 
            uni2ts.module.position.attn_bias.BinaryAttentionBias, 
            uni2ts.module.position.additive.LearnedEmbedding
        )

        for mn, module in self.model.named_modules():
            for pn, param in module.named_parameters():
                if not param.requires_grad:
                    continue
                fpn = f"{mn}.{pn}" if mn else pn
                if pn.endswith("bias"):
                    no_decay.add(fpn)
                elif pn.endswith("weight") and isinstance(module, whitelist_params):
                    decay.add(fpn)
                elif pn.endswith("weight") and isinstance(module, blacklist_params):
                    no_decay.add(fpn)

        param_dict = {pn: p for pn, p in self.model.named_parameters() if p.requires_grad}
        optim_groups = [
            {
                "params": [param_dict[pn] for pn in sorted(list(decay))],
                "weight_decay": weight_decay,
            },
            {
                "params": [param_dict[pn] for pn in sorted(list(no_decay))],
                "weight_decay": 0.0,
            },
        ]

        optimizer = torch.optim.AdamW(optim_groups, lr=self.optim.lr.lr)
    else: 
        self.mssg.print("Optimizer already configured.")
    # Configure Learning Rate scheduler
    try:
        scheduler = None
        if self.optim.lr.flag:
            self.mssg.print("Get scheduler")
            steps_per_epoch     = len(dl_train)
            scheduler = moirai_opt.get_scheduler(
                #name = uni2ts.optim.lr_scheduler.SchedulerType[self.optim.lr.name.upper()],
                name                        = self.optim.lr.name.lower(),
                optimizer                   = optimizer,
                num_warmup_steps            = self.optim.lr.num_warmup_steps,
                num_training_steps          = self.num_epochs*steps_per_epoch,
                scheduler_specific_kwargs   = self.scheduler_specific_kwargs
            )
    # Check errors
    except Exception as e:
        scheds = [ "linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup", "inverse_sqrt", "reduce_lr_on_plateau"]
        self.mssg.print_error(
            f"Please check that lr_scheduler_name is one of the following: {scheds}. Check https://github.com/SalesforceAIResearch/uni2ts/blob/main/src/uni2ts/optim/lr_scheduler.py for updates."
        )
        raise e
    # Setup optimizer & scheduler
    self.optim.optimizer = optimizer
    self.optim.lr.scheduler = scheduler
    self.mssg.final()
    # Restore mssg
    self.mssg.function = func
    self.mssg.level -= 1
Encoder.configure_optimizer_moirai = configure_optimizer_moirai

In [ ]:
#| export
def get_enc_embs_moirai_(# Obtain the embeddings
    self            : Encoder,
    enc_input       : List [ List [ List [ float ] ] ],
    patch_size      : int,
    return_kwargs   : bool,
    to_numpy        : bool = True
):
    return get_enc_embs_moirai(
        enc_input       = enc_input,
        enc_model       = self.model,
        cpu             = self.cpu,
        average_seq_dim = True,
        verbose         = 3, #self.mssg.verbose-self.mssg.level+1,
        to_numpy        = to_numpy,
        patch_size      = patch_size,
        time            = self.time_flag,
        return_kwargs   = return_kwargs
    )

Encoder.get_enc_embs_moirai_ = get_enc_embs_moirai_

In [ ]:
#| export
def get_dist_moirai_(# "Normal" execution
    self            : Encoder,
    enc_input       : List [ List [ List [ float ] ] ],
    patch_size      : int,
    return_kwargs   : bool
):
    return get_dist_moirai(
        enc_input       = enc_input,
        enc_model       = self.model,
        cpu             = self.cpu,
        verbose         = self.mssg.verbose-self.mssg.level+1,
        patch_size      = patch_size,
        time            = self.time_flag,
        return_kwargs   = return_kwargs
    )

Encoder.get_dist_moirai_ = get_dist_moirai_

In [ ]:
#| export
def fine_tune_moirai_eval_step_(
    self        : Encoder, 
    enc_input   : List [ List [ List [ float ] ] ],
    patch_size  : int
):    
    device = "cpu" if self.cpu else torch.cuda.current_device()
    with torch.no_grad():
        embs, moirai_args = self.get_enc_embs_moirai_(enc_input, patch_size, True, False)

        references = enc_input
        mask = moirai_args['observed_mask']
        self.mssg.print_error(f"mask~{mask.shape}, embs~{embs.shape}, references~{references.shape}, data~{self.input.shape}")
        embs = embs * ~mask
        references = references * ~mask 
        embs = embs.to(device)
        references = references.to(device)
        ## -- ver si hay que meter algun preprocesado de tamaños
        for metric in self.metrics:
            metric.add_batch(predictions = embs, references = references)
    return self.metrics
        
Encoder.fine_tune_moirai_eval_step_ = fine_tune_moirai_eval_step_

In [ ]:
#| export
def fine_tune_moirai_eval_(
    self    : Encoder,
    dl_eval,
    eval_results_dict,
    patch_size = 8
):    
    # Select device
    device = "cpu" if self.cpu else torch.cuda.current_device()
    self.model.to(device)
    num_evaluation_steps = len(dl_eval)
    self.model.eval()
    progress_bar = tqdm(range(num_evaluation_steps))
    for batch in dl_eval:
        batch = batch.to(device)
        self.fine_tune_moirai_eval_step_(
            enc_input   = batch,
            patch_size  = patch_size
        )
        progress_bar.update(1)
    progress_bar.close()
    for name in self.metrics_dict.keys():
        metric                  = self.metrics[name]['metric']
        args                    = self.metrics[name]['args']
        eval_results_dict[name] = metric.compute(**args)
    self.model.train()
    return eval_results_dict

Encoder.fine_tune_moirai_eval_ = fine_tune_moirai_eval_

In [ ]:
#| export 
def fine_tune_moirai_train_loop_step_(
    self        : Encoder,
    enc_input   : List[ List [ List [ float ] ] ],
    patch_size  : int 
):
    loss = np.nan
    #device = "cpu" if self.cpu else torch.cuda.current_device()
    #embs, moirai_args = self.get_enc_embs_moirai_(enc_input, patch_size, True)
    #mask = moirai_args['observed_mask']
    loss_func = uni2ts.loss.packed.distribution.PackedNLLLoss()
    distr, args = get_dist_moirai_(enc_input, patch_size, True)
    loss = loss_func(pred = distr,**args)
    return loss 

In [ ]:
#| export 
def fine_tune_moirai_train_(
    self                : Encoder,
    dl_train            : DataLoader
): 
    func = self.mssg.function
    self.mssg.level += 1
    self.mssg.initial_(ut.funcname())
    losses  = []
    # Training loop
    self.mssg.print("Training loop")
    # Mask
    num_training_steps = self.num_epochs*len(dl_train)
    progress_bar = tqdm(range(num_training_steps))
    for epoch in range(self.num_epochs):
        for i, batch in enumerate(dl_train):
            self.mssg.print(f"batch {i}~{batch.shape} | epoch {epoch} | train step {i+epoch} / {num_training_steps}")
            loss = fine_tune_moirai_train_loop_step_()
            self.mssg.print(f"batch {i}~{batch.shape} | epoch {epoch} | After train step {i+epoch} / {num_training_steps}")
            if hasattr(loss, 'item'):
                losses.append(loss)
            else: 
                losses.append(loss.item())
                loss.backward()
            self.optim.optimizer.zero_grad()
            self.optim.optimizer.step()
            if self.optim.lr.flag:
                self.optim.lr.scheduler.step()
            progress_bar.update(1)
    progress_bar.close() 
    self.mssg.final()
    self.mssg.function = func
    self.mssg.level -= 1
Encoder.fine_tune_moirai_train_ = fine_tune_moirai_train_

In [ ]:
#| export
def fine_tune_moirai_single_(
    self            : Encoder,
    eval_pre        : bool  = False,
    eval_post       : bool  = False,
    shot            : bool  = False,
    show_plot       : bool  = False,
    sample_id       : int   = 0,
    patch_size      : int   = 8,
    register_errors  : bool  = True
):
    self.mssg.level+=1
    func = self.mssg.function
    self.mssg.initial_(ut.funcname())
    #--- Setup object parameters ---
    self.show_plot = self.show_plot if show_plot is None else show_plot
    #--- Setup output values
    # Time
    t_shot = 0
    t_eval_1 = 0
    t_eval_2 = 0
    # Training losses
    losses = [],
    # Evaluation metrics & results
    eval_results_pre = {k:[] for k in self.metrics_dict}
    eval_results_post = {k:[] for k in self.metrics_dict}
    # Computation device
    device = "cpu" if self.cpu else torch.cuda.current_device()
    
    #### -------- Start -------####
    if self.time_flag : timer = ut.Time(mssg = self.mssg)
    self.model.to(device)
    self.mssg.print(f"Model Class {self.model.__class__} | Type: {type(self.model)}")
    # Get dataloaders
    self.mssg.print(f"fine_tune_moirai_single | Prepare the dataset | X ~ {self.input.data[sample_id].shape}")
    ( 
        dl_eval, dl_train, 
        ds_eval, ds_train 
    ) = set_train_and_eval_dataloaders(sample_id)
    
    # Setup optimizer
    if self.optim.optimizer is None:
        self.configure_optimizer_moirai(dl_train)
    try:
        ##-- Eval - Pre
        if eval_pre:
            self.mssg.print(f"Eval Pre | wlen {self.input.data[sample_id].shape[2]}")
            if self.time_flag: timer.start()
            eval_results_pre = self.fine_tune_moirai_eval_(dl_eval, eval_results_pre, patch_size)
            if self.time_flag:
                timer.end()
                t_eval_1 = timer.duration()
                timer.show(verbose = self.mssg.verbose)
        ##-- Train
        if shot:
            if self.time_flag: timer.start()
            self.mssg.print(f"Train | wlen {self.input.data[sample_id].shape[2]}")
            losses, self.model = fine_tune_moirai_train_()
            if self.time_flag: 
                timer.end()
                t_shot = timer.duration()
                timer.show(verbose = self.mssg.verbose)

        ##-- Eval - Post
        if eval_post:
            self.mssg.print(f"Eval Post | wlen {self.input.data[sample_id].shape[2]}")
            if self.time_flag: timer.start()
            eval_results_post = self.fine_tune_moirai_eval_(dl_eval, eval_results_post, patch_size)
            if self.time_flag:
                timer.end()
                t_eval_2 = timer.duration()
                timer.show(verbose = self.mssg.verbose)
    except Exception as e:
        if register_errors:
            self.mssg.print_error("Registering error in DataFrame")
            error = {"window": self.input.data[sample_id].shape[2], "error": e}
            self.errors = pd.concat([self.errors, pd.DataFrame([error])])
        else: 
            raise(e)
    self.mssg.final()   
    self.mssg.level += 1
    self.mssg.function = func
    return losses, eval_results_pre, eval_results_post, t_shot, t_eval_1, t_eval_2, self.model
Encoder.fine_tune_moirai_single_ = fine_tune_moirai_single_

In [ ]:
#| export
def fine_tune_moirai_(
    self      : Encoder, 
    eval_pre  : bool = False, 
    eval_post : bool = False, 
    shot      : bool = False,
    time_flag : bool = None,
    show_plot : bool = False,
    register_errors : bool = False
):  
    self.mssg.initial(ut.funcname())
    self.time_flag = self.time_flag if time_flag is None else time_flag
    # Return values
    lossess             = []
    eval_results_pre    = {}
    eval_results_post   = {}
    t_shots             = []
    t_shot              = 0
    t_evals             = []
    t_eval              = 0
    
    # Computation device
    device = "cpu" if self.cpu else torch.cuda.current_device()
    
    # Metrics
    if self.metrics is None:
        self.metrics        = [ EvalMAE ]
        self.metrics_names  = ['mae']
        self.metrics_args   = [{'squared':False}]
        self.metrics_dict   = {
            'mae': {
                'metric': self.metrics[0], 
                'args'  : self.metrics_args[0]
            }
        }
    
    ### ----- Start ----
    if self.input._data.__class__ != ut.WindowedDataset and self.input.size is None:
        self.mssg.print(f"Windows: {len(self.input._data)}")
        raise ValueError(f"Invalid number of windows: {self.input.size}")

    self.mssg.print(f"Processing {self.input.size} datasets : {self.input.shape}")

    self.model.to(device)
    
    
    
    for i in range (self.input.size):
        self.mssg.print(f"Processing wlen {self.input.shape[2]}")
        ( 
            losses, eval_results_pre_, eval_results_post_, t_shot_, t_eval_1, t_eval_2, self.model
        ) =  self.fine_tune_moirai_single_(eval_pre, eval_post, shot, sample_id = i, show_plot = show_plot, register_errors = register_errors)
    
        lossess.append(losses)
        if (eval_pre): eval_results_pre = eval_results_pre_
        
        if (eval_post): 
            if eval_results_post == {}: eval_results_post = {key:[] for key in eval_results_post_.keys()}
            #self.mssg.print_error(f"About to concat {eval_results_post_} to {eval_results_post}")
            for key in eval_results_post.keys():
                eval_results_post[key] += eval_results_post_[key]
            #self.mssg.print_error(f"After concat: {eval_results_post}")
        t_shots.append(t_shot_)
        if eval_pre: t_evals.append(t_eval_1)
        if eval_post: t_evals.append(t_eval_2)
        eval_pre = False
    t_shot = sum(t_shots)
    t_eval = sum(t_evals)
    self.mssg.final(ut.funcname())

    if register_errors: 
        return losses, eval_results_pre, eval_results_post, t_shots, t_shot, t_evals, t_eval, self.model, self.errors
    return lossess, eval_results_pre, eval_results_post, t_shots, t_shot, t_evals, t_eval, self.model

Encoder.fine_tune_moirai_ = fine_tune_moirai_

### Global method

TODO: Revisar que todos los fine-tune fijen primero el porcentaje del dataset / de las ventanas que se utilizan para el conjunto de validación. Primero se aparta el de validación y luego con AL RESTO se le aplica el porcentaje de "dataset_percent". De tal manera que Dataset percent no es el porcentaje del total, sino que tenemos

df ~ TOTAL
df_test ~ TOTAL*validation_percent
df_train ~ ((1-validation_percent)*TOTAL)*training_percent

In [ ]:
#| export
def fine_tune(
    # Optional parameters
    ## Encoder Input
    ## -- Using all parammeters
    X                               : Optional [ Union [ List [ List [ List [ float ]]], List [ float ], pd.DataFrame ] ],
    stride                          : Optional [ int ]              = None,
    batch_size                      : Optional [ int ]              = None,
    n_windows                       : Optional [ int ]              = None,
    n_windows_percent               : Optional [ float ]            = None,
    validation_percent              : Optional [ float ]            = None, 
    training_percent                : Optional [ float ]            = None,
    window_mask_percent             : Optional [ float ]            = None,
    window_sizes                    : Optional [ List [int] ]       = None,
    n_window_sizes                  : Optional [ int ]              = 1,
    window_sizes_offset             : Optional [ int ]              = 0.05,
    windows_min_distance            : Optional [ int ]              = 1,
    full_dataset                    : Optional [ bool ]             = False,
    ##-- Given by Type  
    enc_input                       : Optional [ EncoderInput ]     = None,
    # Optimizer
    optim                           : Optional [ EncoderOptimizer ] = None,
    ## -- Using all parameters
    criterion                       : Optional [ _Loss ]            = torch.nn.MSELoss, 
    optimizer                                                       = None, 
    lr                              : Optional [ float ]            = 5e-5, #1e-4, 
    lr_scheduler_flag               : Optional [ bool ]             = False, 
    lr_scheduler_name               : Optional [ str ]              = "linear",
    lr_scheduler_num_warmup_steps   : Optional [ int ]              = None,
    # Mssg
    ## -- Using all parameters
    verbose                         : Optional[ int ]               = 0, 
    print_to_path                   : Optional[ bool ]              = False,
    print_both                      : Optional[bool]                = False,
    print_path                      : Optional[ str ]               = "~/data/logs/logs.txt",
    print_mode                      : Optional[ str ]               = 'a',
    ## -- Using Type
    mssg                            : Optional [ ut.Mssg ]          = None,
    ## Encoder 
    enc                             : Optional [ Encoder ]          = None,
    ## -- Using all parameters
    num_epochs                      : Optional [ int]               = 3,
    enc_learn                       : Optional [Learner]            = None, 
    cpu                             : Optional [ bool ]             = False,
    to_numpy                        : Optional [ bool ]             = True,
    #- Only for moment
    use_moment_masks                : Optional [ bool ]             = False,
    #- Masking options
    mask_stateful                   : Optional [ bool ]             = False,
    mask_future                     : Optional [ bool ]             = False,
    mask_sync                       : Optional [ bool ]             = False,
    # Non-Optional parameters
    time_flag                       : bool                          = False,
    shot                            : bool                          = True, 
    eval_pre                        : bool                          = True, 
    eval_post                       : bool                          = True,
    use_wandb                       : bool                          = None,
    analysis_mode                   : str                           = None,
    norm_by_sample                  : bool                          = None,
    norm_use_single_batch           : bool                          = None,
    show_plot                       : bool                          = False,
    # Optional metrics
    metrics                                                         = None,
    metrics_names                   : List [ str ]                  = None,
    metrics_args                    : List [ AttrDict ]             = None,
    metrics_dict                    : AttrDict                      = None,
    scheduler_specific_kwargs       : AttrDict                      = None,
    register_errors                 : bool                          = True,
    save_best_or_last               : bool                          = True, #If true, save best else save last
    mix_windows                     : bool                          = True
): 
    enc = _set_encoder(
        X                               = X,
        stride                          = stride,
        batch_size                      = batch_size,
        n_windows                       = n_windows,
        n_windows_percent               = n_windows_percent,
        validation_percent              = validation_percent,
        training_percent                = training_percent,
        window_mask_percent             = window_mask_percent,
        window_sizes                    = window_sizes,
        n_window_sizes                  = n_window_sizes,
        window_sizes_offset             = window_sizes_offset,
        windows_min_distance            = windows_min_distance,
        full_dataset                    = full_dataset,
        enc_input                       = enc_input,
        optim                           = optim,
        criterion                       = criterion,
        optimizer                       = optimizer,
        lr                              = lr,
        lr_scheduler_flag               = lr_scheduler_flag,
        lr_scheduler_name               = lr_scheduler_name,
        lr_scheduler_num_warmup_steps   = lr_scheduler_num_warmup_steps,
        verbose                         = verbose,
        print_to_path                   = print_to_path,
        print_both                      = print_both,
        print_path                      = print_path,
        print_mode                      = print_mode,
        mssg                            = mssg,
        enc                             = enc,
        num_epochs                      = num_epochs,
        enc_learn                       = enc_learn,
        cpu                             = cpu,
        to_numpy                        = to_numpy,
        mask_stateful                   = mask_stateful,
        mask_future                     = mask_future,
        mask_sync                       = mask_sync,
        metrics                         = metrics,
        metrics_names                   = metrics_names,
        metrics_args                    = metrics_args,
        metrics_dict                    = metrics_dict,
        scheduler_specific_kwargs       = scheduler_specific_kwargs,
        mix_windows                     = mix_windows
    )
    func = enc.mssg.function
    enc.mssg.initial_("fine_tune")
    #enc.mssg.print(f"Original enc_learn { enc_learn }  | Final model { enc.model }")
    enc.set_fine_tune_()
    match enc.fine_tune_.__name__:
        case "fine_tune_moment_":
            enc.mssg.print("Use fine_tune_moment parameters")
            result = enc.fine_tune_(
                eval_pre, eval_post, shot, time_flag, use_moment_masks, register_errors, save_best_or_last, mix_windows
            )
        case "fine_tune_mvp_":
            enc.mssg.print("Use fine_tune_mvp parameters")
            result = enc.fine_tune_(eval_pre, eval_post, shot, time_flag, use_wandb = use_wandb, analysis_mode = analysis_mode, norm_by_sample = norm_by_sample, norm_use_single_batch = norm_use_single_batch, show_plot = show_plot)
        case _:
            enc.mssg.print("Use generic fine_tune parameters")
            result = enc.fine_tune_(eval_pre = eval_pre, eval_post = eval_post, shot = shot, time_flag = time_flag)
    enc.mssg.final()
    enc.mssg.function = func
    return result

In [ ]:
#| hide
beep(1)
beep(1)
beep(1)
beep(1)
beep(1)